# 프로젝트명 : 4주차 제조데이터 실습코드

### 📌 프로젝트 소개

- 다이캐스팅은 액체화된 금속을 주조(틀, Frame)에 넣고 원하는 모양의 금속부품을 생산하는 방법입니다.
- 다이캐스팅에서는 주요 4대 요소인 압력, 속도, 시간, 온도에 대해서 조건관리를 중요 시 합니다.

### 📝 현장에서 온 정보
- 데이터 제공기업의 경우 일일 또는 주간 단위로 품질 이슈 현황을 파악하고 있으며 불량원인을 수작업으로 분석하고 있습니다.
- 각 불량에 대한 발생원인과 대책이 정의되어 있으나 이를 적용하여 해결하지 못하고 있는 실정입니다.
- 대부분의 중소기업에서는 관리자 및 작업자의 경험에 의해 설비를 운용하고 있어 체계적인 관리를 하지 못하고 있기 때문입니다.
- 따라서 일정한 공정 환경 및 공정 변수를 관리해 불량에 대응하는 것이 필요합니다.


#### 다이캐스팅

![다이캐스팅](https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/%E1%84%83%E1%85%A1%E1%84%8B%E1%85%B5%E1%84%8F%E1%85%A2%E1%84%89%E1%85%B3%E1%84%90%E1%85%B5%E1%86%BC.gif?alt=media&token=70587460-34c3-4a67-a056-f7a5e6ad8521)

### 라이브러리 불러오기

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import xgboost
import lightgbm
import warnings
from eli5.sklearn import PermutationImportance
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from lightgbm import LGBMClassifier, plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv("https://firebasestorage.googleapis.com/v0/b/ls-storage-e452a.appspot.com/o/casting.csv?alt=media&token=54fffc33-03ad-4431-8e20-34ede32c043f", 
                      encoding='cp949',
                      index_col=0)

### EDA 툴

In [3]:
'''
## Data EDA tool
import sweetviz as sv
## sweetviz를 이용해 데이터를 분석해보자
train_report = sv.analyze(data, 
                          pairwise_analysis='on' #  on: 모든 변수간의 관계를 분석 
                                                 # off: 모든 변수간의 관계를 분석하지 않음
                          )
train_report.show_html('data_report.html')
'''

"\n## Data EDA tool\nimport sweetviz as sv\n## sweetviz를 이용해 데이터를 분석해보자\ntrain_report = sv.analyze(data, \n                          pairwise_analysis='on' #  on: 모든 변수간의 관계를 분석 \n                                                 # off: 모든 변수간의 관계를 분석하지 않음\n                          )\ntrain_report.show_html('data_report.html')\n"

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92015 entries, 0 to 92014
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   line                          92015 non-null  object 
 1   name                          92015 non-null  object 
 2   mold_name                     92015 non-null  object 
 3   time                          92015 non-null  object 
 4   date                          92015 non-null  object 
 5   count                         92015 non-null  int64  
 6   working                       92014 non-null  object 
 7   emergency_stop                92014 non-null  object 
 8   molten_temp                   89754 non-null  float64
 9   facility_operation_cycleTime  92015 non-null  int64  
 10  production_cycletime          92015 non-null  int64  
 11  low_section_speed             92014 non-null  float64
 12  high_section_speed            92014 non-null  float64
 13  m

### 각 변수 탐구 & 전처리

| **성질 그룹**       | **변수 이름**                | **설명**                                                  | **자료형**      |
|---------------------|------------------------------|-----------------------------------------------------------|-----------------|
| 식별 및 시간 정보   | line                         | 생산 라인 정보                                            | object          |
|                     | name                         | 제품 이름                                                 | object          |
|                     | mold_name                    | 금형 이름                                                 | object          |
|                     | time                         | 작업 시간                                                 | object          |
|                     | date                         | 작업 날짜                                                 | object          |
|                     | registration_time            | 등록 시간                                                 | object          |
|                     | mold_code                    | 금형 코드                                                 | int64           |
| 작업 상태 및 이벤트 | working                      | 작업 여부                                                 | object          |
|                     | emergency_stop               | 비상 정지 여부                                            | object          |
|                     | tryshot_signal               | 시도 주입 신호                                            | object          |
| 공정 관련 변수      | count                        | 생산 수량                                                 | int64           |
|                     | facility_operation_cycleTime | 시설 운영 주기 시간                                       | int64           |
|                     | production_cycletime         | 생산 주기 시간                                            | int64           |
|                     | EMS_operation_time           | EMS 작동 시간                                             | int64           |
| 온도 관련 변수      | molten_temp                  | 용탕 온도                                                 | float64         |
|                     | upper_mold_temp1             | 상부 금형 온도 1                                          | float64         |
|                     | upper_mold_temp2             | 상부 금형 온도 2                                          | float64         |
|                     | upper_mold_temp3             | 상부 금형 온도 3                                          | float64         |
|                     | lower_mold_temp1             | 하부 금형 온도 1                                          | float64         |
|                     | lower_mold_temp2             | 하부 금형 온도 2                                          | float64         |
|                     | lower_mold_temp3             | 하부 금형 온도 3                                          | float64         |
|                     | sleeve_temperature           | 슬리브 온도                                               | float64         |
|                     | Coolant_temperature          | 냉각수 온도                                               | float64         |
| 속도 및 압력 관련 변수| low_section_speed            | 낮은 구간 속도                                            | float64         |
|                     | high_section_speed           | 높은 구간 속도                                            | float64         |
|                     | cast_pressure                | 주조 압력                                                 | float64         |
| 품질 및 물리적 특성 관련 변수 | physical_strength        | 물리적 강도                                               | float64         |
|                     | biscuit_thickness            | 비스킷 두께                                               | float64         |
| 용량 및 부피 관련 변수| molten_volume                | 용탕 용량                                                 | float64         |
| 기타 변수           | heating_furnace              | 가열로 정보                                               | object          |
| 목표 변수           | passorfail                   | 합격 또는 불합격 여부                                      | float64         |


#### 고윳값이 1인 열 삭제
- 열 삭제: 고윳값이 1개로 똑같음 - 따라서 주어진 데이터 내에서 비교 불가
    - **line**: 생산 라인 정보. 어떤 생산 라인에서 작업이 이루어졌는지를 나타냅니다.
    - **name**: 제품 이름. 생산된 제품의 이름을 나타냅니다.
    - **mold_name**: 금형 이름. 사용된 금형의 이름을 나타냅니다.

In [5]:
# 'data' 데이터프레임에서 'line', 'name', 'mold_name' 열을 삭제합니다.
data_CK = data.drop(columns=['line', 'name', 'mold_name'])


In [6]:
# 'passorfail' 열의 결측치가 있는 행 제거

data_CK = data_CK.dropna(subset=['passorfail'])

이외에 고윳값이 1개있는 열 확인
: 이전에 위에서 passorfail이 널값인게 유일하게 emergency stop 이 널값이었음

In [7]:
# 고유값이 1개인 열들을 찾기 위해 각 열의 고유값 수를 계산
unique_value_counts = data_CK.apply(lambda x: x.nunique(dropna=False))

# 고유값이 1개인 열들을 필터링
single_unique_columns = unique_value_counts[unique_value_counts == 1].index.tolist()
single_unique_columns

['emergency_stop']

In [8]:
# 위의 emergency_stop 삭제
data_CK = data_CK.drop(columns=single_unique_columns)


In [9]:
data_CK.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92014 entries, 0 to 92014
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   time                          92014 non-null  object 
 1   date                          92014 non-null  object 
 2   count                         92014 non-null  int64  
 3   working                       92014 non-null  object 
 4   molten_temp                   89753 non-null  float64
 5   facility_operation_cycleTime  92014 non-null  int64  
 6   production_cycletime          92014 non-null  int64  
 7   low_section_speed             92014 non-null  float64
 8   high_section_speed            92014 non-null  float64
 9   molten_volume                 46884 non-null  float64
 10  cast_pressure                 92014 non-null  float64
 11  biscuit_thickness             92014 non-null  float64
 12  upper_mold_temp1              92014 non-null  float64
 13  u

남은 변수들

| 번호 | 변수명                        | Non-Null Count | 데이터 타입 | 설명                                   |
|------|-------------------------------|----------------|--------------|----------------------------------------|
| 1    | time                          | 92014          | object       | 작업 시간                              |
| 2    | date                          | 92014          | object       | 작업 날짜                              |
| 3    | count                         | 92014          | int64        | 개수                                   |
| 4    | working                       | 92014          | object       | 작동 여부                              |
| 5    | molten_temp                   | 89753          | float64      | 용융 온도                              |
| 6    | facility_operation_cycleTime  | 92014          | int64        | 시설 작동 주기 시간                   |
| 7    | production_cycletime          | 92014          | int64        | 생산 주기 시간                         |
| 8    | low_section_speed             | 92014          | float64      | 저부 속도                              |
| 9    | high_section_speed            | 92014          | float64      | 고부 속도                              |
| 10   | molten_volume                 | 46884          | float64      | 용융량                                 |
| 11   | cast_pressure                 | 92014          | float64      | 주조 압력                              |
| 12   | biscuit_thickness             | 92014          | float64      | 비스킷 두께                           |
| 13   | upper_mold_temp1              | 92014          | float64      | 상금형 온도1                           |
| 14   | upper_mold_temp2              | 92014          | float64      | 상금형 온도2                           |
| 15   | upper_mold_temp3              | 91702          | float64      | 상금형 온도3                           |
| 16   | lower_mold_temp1              | 92014          | float64      | 하금형 온도1                           |
| 17   | lower_mold_temp2              | 92014          | float64      | 하금형 온도2                           |
| 18   | lower_mold_temp3              | 91702          | float64      | 하금형 온도3                           |
| 19   | sleeve_temperature            | 92014          | float64      | 슬리브 온도                           |
| 20   | physical_strength             | 92014          | float64      | 물리적 강도                           |
| 21   | Coolant_temperature           | 92014          | float64      | 냉각수 온도                           |
| 22   | EMS_operation_time            | 92014          | int64        | EMS 작동 시간                         |
| 23   | registration_time             | 92014          | object       | 등록 시간                              |
| 24   | passorfail                    | 92014          | float64      | 합격 여부                              |
| 25   | tryshot_signal                | 1919           | object       | 시험 발사 신호                         |
| 26   | mold_code                     | 92014          | int64        | 금형 코드                              |
| 27   | heating_furnace               | 42869          | object       | 가열로                                 |


#### 🍪금형 코드

In [10]:
# 금형 코드 별 불량률 계산

# 'mold_code'를 기준으로 그룹화합니다.
grouped_by_mold = data_CK.groupby('mold_code')

# 각 그룹 내에서 합격(0)과 불합격(1)의 개수를 세고, 전체 데이터 수로 나누어 불량률을 계산합니다.
def calculate_defect_rate(group):
    total_count = len(group)
    fail_count = (group['passorfail'] == 1).sum()
    fail_rate = (fail_count / total_count) * 100  # 불량률을 퍼센트로 변환합니다.
    return fail_rate

# 각 금형 코드에 대한 불량률을 계산합니다.
defect_rate_by_mold = grouped_by_mold.apply(calculate_defect_rate)

# 결과를 출력합니다.
print(defect_rate_by_mold)

mold_code
8412    3.465139
8413    3.745442
8573    4.376824
8576    5.044136
8600    5.067568
8722    5.426975
8917    4.332733
dtype: float64


일단 mold_code가 금형에 관계가 있는지 보고 싶어서 7개를 원핫 인코딩

In [11]:
# mold code를 원-핫 인코딩합니다.
data_encoded = pd.get_dummies(data_CK, columns=['mold_code'], prefix='mold')

# 결과를 확인합니다.
print(data_encoded.head())

         time      date  count working  molten_temp  \
0  2019-01-02  16:45:06    258      가동        731.0   
1  2019-01-02  16:45:08    243      가동        720.0   
2  2019-01-02  16:45:58    244      가동        721.0   
3  2019-01-02  16:48:03    245      가동        721.0   
4  2019-01-02  16:50:08    246      가동        721.0   

   facility_operation_cycleTime  production_cycletime  low_section_speed  \
0                           119                   120              110.0   
1                            98                   125              109.0   
2                            98                   122              109.0   
3                           100                   125              112.0   
4                            99                   123              109.0   

   high_section_speed  molten_volume  ...  passorfail  tryshot_signal  \
0               112.0           75.0  ...         0.0             NaN   
1               109.0            NaN  ...         0.0             

#### 🌡️ 온도 변수
- 온도 변수 column :  'molten_temp', 'upper_mold_temp1', 'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3', 'sleeve_temperature', 'Coolant_temperature'  

- 각 변수 설명  
    molten_temp (용융 온도): 금속이 용융되는 온도입니다. 금속이 용융되기 위해서는 특정 온도가 필요합니다. 이 변수는 금속이 용융되는 온도를 나타냅니다.

    upper_mold_temp1, upper_mold_temp2, upper_mold_temp3 (상금형 온도1, 2, 3): 상금형은 금속을 주입하고 응고시키는 역할을 합니다. 이 변수들은 상금형의 온도를 나타냅니다. 주로 상금형의 여러 부분의 온도를 측정하여 다수의 센서로부터 얻은 온도 데이터입니다.

    lower_mold_temp1, lower_mold_temp2, lower_mold_temp3 (하금형 온도1, 2, 3): 하금형은 주입된 금속을 응고시키는 역할을 합니다. 이 변수들은 하금형의 온도를 나타냅니다. 상금형과 마찬가지로 여러 부분의 온도를 측정하여 다수의 센서로부터 얻은 온도 데이터입니다.

    sleeve_temperature (슬리브 온도): 슬리브는 금속이 주입되는 동안 금속이 슬리브를 통과하는 동안 금속을 주입하고 응고시키는 역할을 합니다. 이 변수는 슬리브의 온도를 나타냅니다.

    Coolant_temperature (냉각수 온도): 금속 주조 공정 중에 사용되는 냉각수의 온도입니다. 냉각수는 금속이 응고되는 데 도움을 줍니다.  

- 상금형 vs 하급형:  
  
    상금형 (Upper Mold)의 온도:  
    상금형은 주로 금속이 주입되는 부분으로, 주로 금속이 주입되는 시점에서 금속의 온도와 형태를 제어하기 위해 높은 온도를 유지하는 경향이 있습니다.
    따라서 상금형은 일반적으로 높은 온도로 유지되며, 이는 금속이 형성되는 동안 금속의 응고 속도를 조절하는 데 중요합니다.  
      
    하금형 (Lower Mold)의 온도:  
    하금형은 주로 응고된 금속이 형성되는 부분으로, 주로 응고된 금속의 온도와 형태를 제어하기 위해 낮은 온도를 유지하는 경향이 있습니다.
    따라서 하금형은 일반적으로 더 낮은 온도로 유지되며, 이는 금속이 응고되는 동안 금속의 응고 속도를 조절하는 데 중요합니다.

- 다른 온도변수들:  
  
    용융 온도 (Molten Temperature):  
    용융 온도는 주로 금속이 용융되는 온도를 나타냅니다.  
    금속이 용융되는 온도는 주조 공정에서 가장 중요한 변수 중 하나입니다. 용융 온도는 금속의 녹는 점을 넘어서야만 금속이 융해되고 주조 공정이 진행될 수 있습니다.  
    적절한 용융 온도를 유지함으로써 제품의 품질과 형성에 영향을 미칠 수 있습니다.    

    슬리브 온도 (Sleeve Temperature):  
    슬리브 온도는 슬리브 부분의 온도를 나타냅니다. 슬리브는 금속이 주입되는 동안 금속이 슬리브를 통과하는 동안의 온도를 제어하는 데 사용됩니다.  
    슬리브 온도는 금속이 슬리브를 통과하는 속도와 온도를 제어하여 금속의 형태와 속성을 조절하는 데 중요한 역할을 합니다.  
    적절한 슬리브 온도를 유지함으로써 금속의 흐름과 응고 속도를 조절할 수 있으며, 이는 제품의 형성 및 품질에 영향을 미칠 수 있습니다.  

    냉각수 온도 (Coolant Temperature):   

    냉각수 온도는 주조 공정 중에 사용되는 냉각수의 온도를 나타냅니다. 냉각수는 금속이 응고되는 데 도움을 주는 역할을 합니다.  
    냉각수는 금속이 응고되는 속도와 금속의 온도를 조절하는 데 사용됩니다. 적절한 냉각수 온도를 유지함으로써 제품의 응고 속도와 품질을 조절할 수 있습니다.  

In [12]:
# molten_temp 열에서 고윳값을 추출합니다.
unique_values = sorted(data_CK['molten_temp'].unique())

# 그룹의 수를 나타내는 변수를 초기화합니다.
group_count = 1

# 연속적이지 않은 곳에 "//"를 넣어서 출력합니다.
for i in range(len(unique_values) - 1):
    print(unique_values[i], end=" ")
    if unique_values[i] != unique_values[i + 1] - 1:
        print("// ", end="")
        group_count += 1
print(unique_values[-1])

# 그룹의 수를 출력합니다.
print("그룹의 수:", group_count)


0.0 // 7.0 // 70.0 71.0 72.0 73.0 // 626.0 // 629.0 630.0 // 632.0 633.0 634.0 635.0 636.0 637.0 638.0 639.0 640.0 641.0 642.0 643.0 644.0 645.0 646.0 647.0 648.0 649.0 650.0 651.0 652.0 653.0 654.0 655.0 656.0 657.0 658.0 659.0 660.0 661.0 662.0 663.0 664.0 665.0 666.0 667.0 668.0 669.0 670.0 671.0 672.0 673.0 674.0 675.0 676.0 677.0 678.0 679.0 680.0 681.0 682.0 683.0 684.0 685.0 686.0 687.0 688.0 689.0 690.0 691.0 692.0 693.0 694.0 695.0 696.0 697.0 698.0 699.0 700.0 701.0 702.0 703.0 704.0 705.0 706.0 707.0 708.0 709.0 710.0 711.0 712.0 713.0 714.0 715.0 716.0 717.0 718.0 719.0 720.0 721.0 722.0 723.0 724.0 725.0 726.0 727.0 728.0 729.0 730.0 731.0 732.0 733.0 734.0 735.0 // nan
그룹의 수: 7


In [13]:
# 각 mold_code의 데이터 개수를 출력합니다.
mold_code_counts = data_CK['mold_code'].value_counts()
print(mold_code_counts)


8917    31112
8412    21471
8722    20048
8573     9596
8413     6034
8600     2960
8576      793
Name: mold_code, dtype: int64


In [14]:
data_CK.columns

Index(['time', 'date', 'count', 'working', 'molten_temp',
       'facility_operation_cycleTime', 'production_cycletime',
       'low_section_speed', 'high_section_speed', 'molten_volume',
       'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
       'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1',
       'lower_mold_temp2', 'lower_mold_temp3', 'sleeve_temperature',
       'physical_strength', 'Coolant_temperature', 'EMS_operation_time',
       'registration_time', 'passorfail', 'tryshot_signal', 'mold_code',
       'heating_furnace'],
      dtype='object')

### mold_code 별

In [15]:
# 각 mold_code에 해당하는 데이터프레임을 변수에 저장합니다.
df_8917 = data_CK[data_CK['mold_code'] == 8917]
df_8412 = data_CK[data_CK['mold_code'] == 8412]
df_8722 = data_CK[data_CK['mold_code'] == 8722]
df_8573 = data_CK[data_CK['mold_code'] == 8573]
df_8413 = data_CK[data_CK['mold_code'] == 8413]
df_8600 = data_CK[data_CK['mold_code'] == 8600]
df_8576 = data_CK[data_CK['mold_code'] == 8576]

# 각 변수에 저장된 데이터프레임을 확인합니다.
print("df_8917:")
print(df_8917.head())

print("df_8412:")
print(df_8412.head())

print("df_8722:")
print(df_8722.head())

print("df_8573:")
print(df_8573.head())

print("df_8413:")
print(df_8413.head())

print("df_8600:")
print(df_8600.head())

print("df_8576:")
print(df_8576.head())


df_8917:
             time      date  count working  molten_temp  \
15236  2019-01-22  08:01:39      3      가동        712.0   
15238  2019-01-22  08:03:41      4      가동        712.0   
15240  2019-01-22  08:05:40      5      가동        713.0   
15242  2019-01-22  08:07:42      6      가동        712.0   
15244  2019-01-22  08:09:44      7      가동        713.0   

       facility_operation_cycleTime  production_cycletime  low_section_speed  \
15236                           121                    80              109.0   
15238                           121                   119              109.0   
15240                           121                   119              110.0   
15242                           119                   119              110.0   
15244                           121                   122              110.0   

       high_section_speed  molten_volume  ...  lower_mold_temp3  \
15236               112.0            NaN  ...            1449.0   
15238               1

#### 8917

In [16]:
# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8917.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹을 확인합니다.
for i, group in enumerate(groups):
    print(f"Group {i+1}:")
    print(group)
    print("\n")

Group 1:
             time      date  count working  molten_temp  \
15236  2019-01-22  08:01:39      3      가동        712.0   
15238  2019-01-22  08:03:41      4      가동        712.0   
15240  2019-01-22  08:05:40      5      가동        713.0   
15242  2019-01-22  08:07:42      6      가동        712.0   
15244  2019-01-22  08:09:44      7      가동        713.0   
...           ...       ...    ...     ...          ...   
15733  2019-01-22  18:52:43    271      가동        712.0   
15735  2019-01-22  18:54:33    272      가동        712.0   
15737  2019-01-22  18:56:41    273      가동        711.0   
15739  2019-01-22  18:58:29    274      가동        711.0   
15741  2019-01-22  19:00:32    275      가동        710.0   

       facility_operation_cycleTime  production_cycletime  low_section_speed  \
15236                           121                    80              109.0   
15238                           121                   119              109.0   
15240                           121       

In [17]:
# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8917.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 count가 연속적인지 확인하는 코드
non_continuous_groups_count = 0
for i, group in enumerate(groups):
    non_continuous_count = 0
    previous_count = None
    
    for index, row in group.iterrows():
        if previous_count is not None and row['count'] != previous_count + 1:
            non_continuous_count += 1
        previous_count = row['count']
    
    print(f"Group {i+1}: 연속적이지 않은 행의 수: {non_continuous_count}")
    
    # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
    if non_continuous_count != 0:
        non_continuous_groups_count += 1

# 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 출력합니다.
print(f"값이 0이 아닌 그룹의 수: {non_continuous_groups_count}")

Group 1: 연속적이지 않은 행의 수: 0
Group 2: 연속적이지 않은 행의 수: 0
Group 3: 연속적이지 않은 행의 수: 1
Group 4: 연속적이지 않은 행의 수: 0
Group 5: 연속적이지 않은 행의 수: 0
Group 6: 연속적이지 않은 행의 수: 0
Group 7: 연속적이지 않은 행의 수: 2
Group 8: 연속적이지 않은 행의 수: 0
Group 9: 연속적이지 않은 행의 수: 0
Group 10: 연속적이지 않은 행의 수: 0
Group 11: 연속적이지 않은 행의 수: 0
Group 12: 연속적이지 않은 행의 수: 0
Group 13: 연속적이지 않은 행의 수: 0
Group 14: 연속적이지 않은 행의 수: 0
Group 15: 연속적이지 않은 행의 수: 0
Group 16: 연속적이지 않은 행의 수: 0
Group 17: 연속적이지 않은 행의 수: 0
Group 18: 연속적이지 않은 행의 수: 0
Group 19: 연속적이지 않은 행의 수: 0
Group 20: 연속적이지 않은 행의 수: 0
Group 21: 연속적이지 않은 행의 수: 0
Group 22: 연속적이지 않은 행의 수: 0
Group 23: 연속적이지 않은 행의 수: 0
Group 24: 연속적이지 않은 행의 수: 0
Group 25: 연속적이지 않은 행의 수: 0
Group 26: 연속적이지 않은 행의 수: 0
Group 27: 연속적이지 않은 행의 수: 0
Group 28: 연속적이지 않은 행의 수: 0
Group 29: 연속적이지 않은 행의 수: 0
Group 30: 연속적이지 않은 행의 수: 1
Group 31: 연속적이지 않은 행의 수: 0
Group 32: 연속적이지 않은 행의 수: 0
Group 33: 연속적이지 않은 행의 수: 0
Group 34: 연속적이지 않은 행의 수: 0
Group 35: 연속적이지 않은 행의 수: 0
Group 36: 연속적이지 않은 행의 수: 0
Group 37: 연속적이지 않은 행의 수: 0
Group 38: 

In [18]:
import pandas as pd

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8917.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 숫자부터 연속적으로 1이 나왔던 마지막 숫자까지를 출력합니다.
group_data = []
for i, group in enumerate(groups):
    start_count = None
    end_count = None
    consecutive_count = 0
    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if start_count is None:
                start_count = row['count']
            end_count = row['count']
            consecutive_count += 1
        elif start_count is not None:
            group_data.append({
                '그룹이름': f'Group {i+1}',
                '처음_passorfail_1': start_count,
                '연속적으로_1이_나왔던_마지막_숫자': end_count,
                '연속적인_1의_개수': consecutive_count
            })
            start_count = None
            end_count = None
            consecutive_count = 0

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(group_data)
result_df.head()


,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,Group 1,3,5,3
1,Group 1,160,160,1
2,Group 1,185,194,10
3,Group 1,227,227,1
4,Group 1,268,268,1


In [19]:
# 각 그룹별 첫 번째 행을 뽑아서 새로운 데이터프레임으로 만듭니다.
first_rows_df = result_df.groupby('그룹이름').first().reset_index()

# 그룹 이름을 순서대로 정렬합니다.
first_rows_df['그룹이름'] = first_rows_df['그룹이름'].str.extract('(\d+)').astype(int)
first_rows_df = first_rows_df.sort_values('그룹이름').reset_index(drop=True)

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

# 결과를 출력합니다.
first_rows_df


,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,1,3,5,3
1,2,1,6,6
2,3,1,5,5
3,4,1,6,6
4,5,1,6,6
5,6,1,7,7
6,7,1,6,6
7,8,1,5,5
8,9,1,5,5
9,10,1,7,7


In [20]:
# first_rows_df의 '처음으로_연속으로_1이_나온_개수' 열의 모든 값을 더합니다.
df_8917_first_drs = first_rows_df['연속적인_1의_개수'].sum()

# df_8917에서 passorfail이 1인 전체 데이터 개수
passorfail_1_count = len(df_8917[df_8917['passorfail'] == 1])

# df_8917의 전체 데이터 개수
total_data_count = len(df_8917)

# passorfail이 1인 전체 데이터 개수 / df_8917의 전체 데이터 개수 * 100 계산
defect_rate = (passorfail_1_count / total_data_count) * 100

# df_8917_first_drs / df_8917에서 passorfail이 1인 전체 데이터 개수 * 100 계산
defect_ratio = (df_8917_first_drs / passorfail_1_count) * 100

# 결과 출력
print("passorfail이 1인 전체 데이터 개수:", passorfail_1_count)
print("df_8917의 전체 데이터 개수:", total_data_count)
print("불량률:", defect_rate)
print("처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율:", defect_ratio)


passorfail이 1인 전체 데이터 개수: 1348
df_8917의 전체 데이터 개수: 31112
불량률: 4.332733350475701
처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율: 45.326409495548965


##### 🤔 연속적이지 않은거는 무슨일이지...

In [21]:
# Group 45 데이터프레임 가져오기
group_45 = groups[44]

# Group 45에서 연속적이지 않은 'count' 값 찾기
previous_count = None
non_continuous_counts = []

for index, row in group_45.iterrows():
    if previous_count is not None and row['count'] != previous_count + 1:
        non_continuous_counts.append(row['count'])
    previous_count = row['count']

# Group 45에서 연속적이지 않았던 'count' 값들 출력
print("Group 45에서 연속적이지 않았던 'count' 값들:")
print(non_continuous_counts)


Group 45에서 연속적이지 않았던 'count' 값들:
[4, 2, 6, 3, 7, 4, 8, 5, 9, 6, 10, 7, 11, 8, 12, 9, 13, 10, 14, 11, 15, 12, 16, 13, 17, 14, 18, 15, 19, 16, 20, 17, 23, 18, 24, 19, 25, 20, 26, 23, 27, 24, 28, 25, 29, 26, 30, 27, 31, 28, 32, 29, 33, 30, 34, 31, 35, 32, 36, 33, 37, 34, 38, 35, 39, 36, 40, 37, 41, 38, 42, 39, 43, 40, 44, 41, 45, 42, 46, 43, 47, 44, 49, 45, 50, 46, 51, 47, 52, 48, 53, 49, 54, 50, 55, 51, 56, 52, 57, 53, 58, 54, 59, 55, 60, 56, 61, 57, 62, 58, 63, 59, 64, 60, 65, 61, 66, 62, 67, 63, 68, 64, 70, 65, 71, 66, 72, 67, 73, 68, 74, 69, 75, 70, 76, 71, 77, 72, 78, 73, 79, 74, 80, 75, 81, 76, 82, 77, 83, 78, 84, 79, 85, 80, 86, 81, 87, 82, 88, 83, 89, 84, 94, 85, 95, 86, 96, 87, 97, 88, 98, 89, 99, 90, 100, 91, 101, 102, 103, 104, 105, 106, 107, 108, 110, 109, 111, 110, 112, 111, 114, 112, 115, 113, 116, 114, 117, 115, 118, 116, 119, 117, 120, 118, 121, 119, 122, 120, 124, 121, 125, 122, 126, 123, 127, 124, 128, 125, 129, 126, 130, 127, 131, 128, 132, 129, 133, 130, 134, 131, 135,

In [22]:
# 인덱스가 짝수인 원소와 홀수인 원소를 담을 리스트 초기화
even_index_elements = []
odd_index_elements = []

# 리스트를 순회하며 인덱스에 따라 원소를 분리
for index, value in enumerate(non_continuous_counts):
    if index % 2 == 0:
        even_index_elements.append(value)
    else:
        odd_index_elements.append(value)

# 연속되지 않는 구간을 찾는 함수
def find_non_continuous_segments(lst):
    non_continuous_segments = []
    for i in range(len(lst) - 1):
        if lst[i] + 1 != lst[i + 1]:
            non_continuous_segments.append(f"{lst[i]}➡️{lst[i + 1]}")
    return non_continuous_segments

# 짝수 인덱스 리스트에서 연속되지 않는 구간 찾기
even_index_non_continuous = find_non_continuous_segments(even_index_elements)

# 홀수 인덱스 리스트에서 연속되지 않는 구간 찾기
odd_index_non_continuous = find_non_continuous_segments(odd_index_elements)

# 결과 출력
print("인덱스가 짝수인 원소 리스트:", even_index_elements)
print("연속되지 않는 구간:", even_index_non_continuous)

print("인덱스가 홀수인 원소 리스트:", odd_index_elements)
print("연속되지 않는 구간:", odd_index_non_continuous)

인덱스가 짝수인 원소 리스트: [4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 94, 95, 96, 97, 98, 99, 100, 101, 103, 105, 107, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 197, 198, 199, 200, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 229, 231, 232, 233, 234, 235, 236, 237, 239, 240, 242, 243, 245, 247, 248, 249, 250, 251, 252,

In [23]:
# Group 47 데이터프레임 가져오기
group_47 = groups[46]

# Group 45에서 연속적이지 않은 'count' 값 찾기
previous_count = None
non_continuous_counts = []

for index, row in group_45.iterrows():
    if previous_count is not None and row['count'] != previous_count + 1:
        non_continuous_counts.append(row['count'])
    previous_count = row['count']

# Group 47에서 연속적이지 않았던 'count' 값들 출력
print("Group 47에서 연속적이지 않았던 'count' 값들:")
print(non_continuous_counts)

Group 47에서 연속적이지 않았던 'count' 값들:
[4, 2, 6, 3, 7, 4, 8, 5, 9, 6, 10, 7, 11, 8, 12, 9, 13, 10, 14, 11, 15, 12, 16, 13, 17, 14, 18, 15, 19, 16, 20, 17, 23, 18, 24, 19, 25, 20, 26, 23, 27, 24, 28, 25, 29, 26, 30, 27, 31, 28, 32, 29, 33, 30, 34, 31, 35, 32, 36, 33, 37, 34, 38, 35, 39, 36, 40, 37, 41, 38, 42, 39, 43, 40, 44, 41, 45, 42, 46, 43, 47, 44, 49, 45, 50, 46, 51, 47, 52, 48, 53, 49, 54, 50, 55, 51, 56, 52, 57, 53, 58, 54, 59, 55, 60, 56, 61, 57, 62, 58, 63, 59, 64, 60, 65, 61, 66, 62, 67, 63, 68, 64, 70, 65, 71, 66, 72, 67, 73, 68, 74, 69, 75, 70, 76, 71, 77, 72, 78, 73, 79, 74, 80, 75, 81, 76, 82, 77, 83, 78, 84, 79, 85, 80, 86, 81, 87, 82, 88, 83, 89, 84, 94, 85, 95, 86, 96, 87, 97, 88, 98, 89, 99, 90, 100, 91, 101, 102, 103, 104, 105, 106, 107, 108, 110, 109, 111, 110, 112, 111, 114, 112, 115, 113, 116, 114, 117, 115, 118, 116, 119, 117, 120, 118, 121, 119, 122, 120, 124, 121, 125, 122, 126, 123, 127, 124, 128, 125, 129, 126, 130, 127, 131, 128, 132, 129, 133, 130, 134, 131, 135,

In [24]:
# 인덱스가 짝수인 원소와 홀수인 원소를 담을 리스트 초기화
even_index_elements = []
odd_index_elements = []

# 리스트를 순회하며 인덱스에 따라 원소를 분리
for index, value in enumerate(non_continuous_counts):
    if index % 2 == 0:
        even_index_elements.append(value)
    else:
        odd_index_elements.append(value)

# 연속되지 않는 구간을 찾는 함수
def find_non_continuous_segments(lst):
    non_continuous_segments = []
    for i in range(len(lst) - 1):
        if lst[i] + 1 != lst[i + 1]:
            non_continuous_segments.append(f"{lst[i]}➡️{lst[i + 1]}")
    return non_continuous_segments

# 짝수 인덱스 리스트에서 연속되지 않는 구간 찾기
even_index_non_continuous = find_non_continuous_segments(even_index_elements)

# 홀수 인덱스 리스트에서 연속되지 않는 구간 찾기
odd_index_non_continuous = find_non_continuous_segments(odd_index_elements)

# 결과 출력
print("인덱스가 짝수인 원소 리스트:", even_index_elements)
print("연속되지 않는 구간:", even_index_non_continuous)

print("인덱스가 홀수인 원소 리스트:", odd_index_elements)
print("연속되지 않는 구간:", odd_index_non_continuous)

인덱스가 짝수인 원소 리스트: [4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 94, 95, 96, 97, 98, 99, 100, 101, 103, 105, 107, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 168, 169, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 197, 198, 199, 200, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 224, 226, 227, 229, 231, 232, 233, 234, 235, 236, 237, 239, 240, 242, 243, 245, 247, 248, 249, 250, 251, 252,

##### 🕵️ t-test통해 첫 연속적인 1, 첫 연속적인 0의 유의미한 변수 찾기 

In [25]:
import pandas as pd
from scipy.stats import ttest_ind

# df_8917로 작업을 예시로 수행합니다.
df = df_8917

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 결과 저장할 리스트
result_list = []

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 그룹과 그 다음 0이 연속되는 그룹으로 나누어 t-test를 수행합니다.
for i, group in enumerate(groups):
    start_count_1 = None
    end_count_1 = None
    start_count_0 = None
    end_count_0 = None
    consecutive_count_1 = 0
    consecutive_count_0 = 0
    found_first_1 = False
    found_first_0 = False

    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if not found_first_1:
                start_count_1 = row['count']
                found_first_1 = True
            end_count_1 = row['count']
            consecutive_count_1 += 1
        elif found_first_1 and not found_first_0:
            start_count_0 = row['count']
            found_first_0 = True
            consecutive_count_0 += 1
            end_count_0 = row['count']
        elif found_first_0 and row['passorfail'] == 0:
            end_count_0 = row['count']
            consecutive_count_0 += 1

    # 그룹 나누기
    group_1 = group[(group['count'] >= start_count_1) & (group['count'] <= end_count_1)]
    group_0 = group[(group['count'] >= start_count_0) & (group['count'] <= end_count_0)]

    # t-test를 수행할 열들
    columns_to_test = ['molten_temp', 'facility_operation_cycleTime', 'production_cycletime', 'low_section_speed',
                       'high_section_speed', 'molten_volume', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
                       'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3',
                       'sleeve_temperature', 'physical_strength', 'Coolant_temperature', 'EMS_operation_time']

    significant_columns = []

    for col in columns_to_test:
        if not group_1[col].dropna().empty and not group_0[col].dropna().empty:
            t_stat, p_value = ttest_ind(group_1[col].dropna(), group_0[col].dropna(), equal_var=False)
            if p_value < 0.05:
                significant_columns.append((col, t_stat, p_value))

    # 결과 저장
    result_list.append({
        'Group': f'Group {i+1}',
        'Significant Columns': significant_columns,
        'Group 1 Means': group_1[columns_to_test].mean(),
        'Group 0 Means': group_0[columns_to_test].mean()
    })

# 결과 출력
for result in result_list:
    print(result['Group'])
    print("Significant Columns:", result['Significant Columns'])
    print("Group 1 Means:")
    print(result['Group 1 Means'])
    print("Group 0 Means:")
    print(result['Group 0 Means'])
    print("\n")

# 모든 그룹에서 유의미한 차이를 보인 공통 열들 확인
common_significant_columns = set(result_list[0]['Significant Columns'])
for result in result_list[1:]:
    common_significant_columns.intersection_update(result['Significant Columns'])

print("All Groups Common Significant Columns:")
print(common_significant_columns)


Group 1
Significant Columns: []
Group 1 Means:
molten_temp                      711.560150
facility_operation_cycleTime     120.251880
production_cycletime             119.744361
low_section_speed                109.827068
high_section_speed               112.011278
molten_volume                           NaN
cast_pressure                    321.281955
biscuit_thickness                 53.011278
upper_mold_temp1                 102.605263
upper_mold_temp2                 162.969925
upper_mold_temp3                1449.000000
lower_mold_temp1                 176.342105
lower_mold_temp2                 142.609023
lower_mold_temp3                1449.000000
sleeve_temperature               477.443609
physical_strength                707.056391
Coolant_temperature               32.383459
EMS_operation_time                23.000000
dtype: float64
Group 0 Means:
molten_temp                      711.548148
facility_operation_cycleTime     120.207407
production_cycletime             119.833333

In [26]:
import pandas as pd
from scipy.stats import ttest_ind

# df_8917로 작업을 예시로 수행합니다.
df = df_8917

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 연속적으로 첫 1이 나온 데이터들을 저장할 리스트
group_1_all = []
# 연속적으로 첫 0이 나온 데이터들을 저장할 리스트
group_0_all = []

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 그룹과 그 다음 0이 연속되는 그룹으로 나눕니다.
for i, group in enumerate(groups):
    start_count_1 = None
    end_count_1 = None
    start_count_0 = None
    end_count_0 = None
    consecutive_count_1 = 0
    consecutive_count_0 = 0
    found_first_1 = False
    found_first_0 = False

    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if not found_first_1:
                start_count_1 = row['count']
                found_first_1 = True
            end_count_1 = row['count']
            consecutive_count_1 += 1
        elif found_first_1 and not found_first_0:
            start_count_0 = row['count']
            found_first_0 = True
            consecutive_count_0 += 1
            end_count_0 = row['count']
        elif found_first_0 and row['passorfail'] == 0:
            end_count_0 = row['count']
            consecutive_count_0 += 1

    # 그룹 나누기
    group_1 = group[(group['count'] >= start_count_1) & (group['count'] <= end_count_1)]
    group_0 = group[(group['count'] >= start_count_0) & (group['count'] <= end_count_0)]

    group_1_all.append(group_1)
    group_0_all.append(group_0)

# 모든 그룹의 데이터를 하나의 데이터프레임으로 합칩니다.
group_1_combined = pd.concat(group_1_all, ignore_index=True)
group_0_combined = pd.concat(group_0_all, ignore_index=True)

# t-test를 수행할 열들
columns_to_test = ['molten_temp', 'facility_operation_cycleTime', 'production_cycletime', 'low_section_speed',
                   'high_section_speed', 'molten_volume', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
                   'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3',
                   'sleeve_temperature', 'physical_strength', 'Coolant_temperature', 'EMS_operation_time']

significant_columns = []

# 각 열에 대해 t-test를 수행하고 유의미한 차이를 보이는 열을 찾습니다.
for col in columns_to_test:
    if not group_1_combined[col].dropna().empty and not group_0_combined[col].dropna().empty:
        t_stat, p_value = ttest_ind(group_1_combined[col].dropna(), group_0_combined[col].dropna(), equal_var=False)
        if p_value < 0.05:
            significant_columns.append((col, t_stat, p_value))

# 결과 출력
print("Significant Columns:")
for col, t_stat, p_value in significant_columns:
    print(f"{col}: t-statistic = {t_stat}, p-value = {p_value}")

print("\nGroup 1 Means:")
print(group_1_combined[columns_to_test].mean())

print("\nGroup 0 Means:")
print(group_0_combined[columns_to_test].mean())


Significant Columns:
molten_temp: t-statistic = 4.474951263377227, p-value = 7.663658518850572e-06
production_cycletime: t-statistic = -2.4889688993243038, p-value = 0.012815160896404299
low_section_speed: t-statistic = -8.927176137806928, p-value = 4.586357521195487e-19
cast_pressure: t-statistic = -20.906880480459712, p-value = 2.5311646156809974e-96
biscuit_thickness: t-statistic = 3.0789415205740998, p-value = 0.002079340362553751
upper_mold_temp1: t-statistic = 4.343652947586796, p-value = 1.4045449408986237e-05
upper_mold_temp2: t-statistic = -12.171278323950038, p-value = 5.119508425122805e-34
lower_mold_temp1: t-statistic = -15.709039972224923, p-value = 1.9249605010170332e-55
lower_mold_temp2: t-statistic = -2.0727715419957464, p-value = 0.03819978256961022
sleeve_temperature: t-statistic = 2.000518908286667, p-value = 0.04545042921668579
physical_strength: t-statistic = -7.3653636823257616, p-value = 1.8162939012371921e-13
Coolant_temperature: t-statistic = -21.06087678985557

#### 8412

In [27]:
# 그룹을 저장할 리스트 초기화
groups_8412 = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8412.iterrows():
    if row['count'] == 1 and current_group:
        groups_8412.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups_8412.append(pd.DataFrame(current_group))

# 각 그룹별로 count가 연속적인지 확인하는 코드
non_continuous_groups_count_8412 = 0
for i, group in enumerate(groups_8412):
    non_continuous_count = 0
    previous_count = None
    
    for index, row in group.iterrows():
        if previous_count is not None and row['count'] != previous_count + 1:
            non_continuous_count += 1
        previous_count = row['count']
    
    print(f"Group {i+1}: 연속적이지 않은 행의 수: {non_continuous_count}")
    
    # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
    if non_continuous_count != 0:
        non_continuous_groups_count_8412 += 1

# 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 출력합니다.
print(f"값이 0이 아닌 그룹의 수: {non_continuous_groups_count_8412}")


Group 1: 연속적이지 않은 행의 수: 0
Group 2: 연속적이지 않은 행의 수: 8
Group 3: 연속적이지 않은 행의 수: 0
Group 4: 연속적이지 않은 행의 수: 0
Group 5: 연속적이지 않은 행의 수: 1
Group 6: 연속적이지 않은 행의 수: 0
Group 7: 연속적이지 않은 행의 수: 1
Group 8: 연속적이지 않은 행의 수: 0
Group 9: 연속적이지 않은 행의 수: 0
Group 10: 연속적이지 않은 행의 수: 0
Group 11: 연속적이지 않은 행의 수: 2
Group 12: 연속적이지 않은 행의 수: 0
Group 13: 연속적이지 않은 행의 수: 0
Group 14: 연속적이지 않은 행의 수: 1
Group 15: 연속적이지 않은 행의 수: 2
Group 16: 연속적이지 않은 행의 수: 0
Group 17: 연속적이지 않은 행의 수: 0
Group 18: 연속적이지 않은 행의 수: 0
Group 19: 연속적이지 않은 행의 수: 0
Group 20: 연속적이지 않은 행의 수: 0
Group 21: 연속적이지 않은 행의 수: 1
Group 22: 연속적이지 않은 행의 수: 0
Group 23: 연속적이지 않은 행의 수: 2
Group 24: 연속적이지 않은 행의 수: 0
Group 25: 연속적이지 않은 행의 수: 0
Group 26: 연속적이지 않은 행의 수: 0
Group 27: 연속적이지 않은 행의 수: 0
Group 28: 연속적이지 않은 행의 수: 0
Group 29: 연속적이지 않은 행의 수: 0
Group 30: 연속적이지 않은 행의 수: 0
Group 31: 연속적이지 않은 행의 수: 0
Group 32: 연속적이지 않은 행의 수: 0
Group 33: 연속적이지 않은 행의 수: 0
Group 34: 연속적이지 않은 행의 수: 0
Group 35: 연속적이지 않은 행의 수: 0
Group 36: 연속적이지 않은 행의 수: 0
Group 37: 연속적이지 않은 행의 수: 1
Group 38: 

In [28]:
import pandas as pd

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8412.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 숫자부터 연속적으로 1이 나왔던 마지막 숫자까지를 출력합니다.
group_data = []
for i, group in enumerate(groups):
    start_count = None
    end_count = None
    consecutive_count = 0
    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if start_count is None:
                start_count = row['count']
            end_count = row['count']
            consecutive_count += 1
        elif start_count is not None:
            group_data.append({
                '그룹이름': f'Group {i+1}',
                '처음_passorfail_1': start_count,
                '연속적으로_1이_나왔던_마지막_숫자': end_count,
                '연속적인_1의_개수': consecutive_count
            })
            start_count = None
            end_count = None
            consecutive_count = 0

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(group_data)
result_df.head()


,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,Group 3,1,7,7
1,Group 3,39,39,1
2,Group 3,46,47,2
3,Group 3,87,87,1
4,Group 3,92,92,1


In [29]:
# 각 그룹별 첫 번째 행을 뽑아서 새로운 데이터프레임으로 만듭니다.
first_rows_df = result_df.groupby('그룹이름').first().reset_index()

# 그룹 이름을 순서대로 정렬합니다.
first_rows_df['그룹이름'] = first_rows_df['그룹이름'].str.extract('(\d+)').astype(int)
first_rows_df = first_rows_df.sort_values('그룹이름').reset_index(drop=True)

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

# 결과를 출력합니다.
first_rows_df

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,3,1,7,7
1,4,1,4,4
2,5,1,5,5
3,6,1,3,3
4,7,1,6,6
5,8,1,3,3
6,9,1,5,5
7,10,1,6,6
8,11,1,2,2
9,12,248,248,1


In [30]:
# first_rows_df의 '처음으로_연속으로_1이_나온_개수' 열의 모든 값을 더합니다.
df_8412_first_drs = first_rows_df['연속적인_1의_개수'].sum()

# df_8412에서 passorfail이 1인 전체 데이터 개수
passorfail_1_count_8412 = len(df_8412[df_8412['passorfail'] == 1])

# df_8412의 전체 데이터 개수
total_data_count_8412 = len(df_8412)

# passorfail이 1인 전체 데이터 개수 / df_8412의 전체 데이터 개수 * 100 계산
defect_rate_8412 = (passorfail_1_count_8412 / total_data_count_8412) * 100

# df_8412_first_drs / df_8412에서 passorfail이 1인 전체 데이터 개수 * 100 계산
defect_ratio_8412 = (df_8412_first_drs / passorfail_1_count_8412) * 100

# 결과 출력
print("8412 passorfail이 1인 전체 데이터 개수:", passorfail_1_count_8412)
print("df_8412의 전체 데이터 개수:", total_data_count_8412)
print("8412 불량률:", defect_rate_8412)
print("8412 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율:", defect_ratio_8412)


8412 passorfail이 1인 전체 데이터 개수: 744
df_8412의 전체 데이터 개수: 21471
8412 불량률: 3.465139024731032
8412 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율: 53.36021505376344


#### 8722

In [31]:
# 그룹을 저장할 리스트 초기화
groups_8722 = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8722.iterrows():
    if row['count'] == 1 and current_group:
        groups_8722.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups_8722.append(pd.DataFrame(current_group))

# 각 그룹별로 count가 연속적인지 확인하는 코드
non_continuous_groups_count_8722 = 0
for i, group in enumerate(groups_8722):
    non_continuous_count = 0
    previous_count = None
    
    for index, row in group.iterrows():
        if previous_count is not None and row['count'] != previous_count + 1:
            non_continuous_count += 1
        previous_count = row['count']
    
    print(f"Group {i+1}: 연속적이지 않은 행의 수: {non_continuous_count}")
    
    # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
    if non_continuous_count != 0:
        non_continuous_groups_count_8722 += 1

# 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 출력합니다.
print(f"값이 0이 아닌 그룹의 수: {non_continuous_groups_count_8722}")


Group 1: 연속적이지 않은 행의 수: 1
Group 2: 연속적이지 않은 행의 수: 4
Group 3: 연속적이지 않은 행의 수: 0
Group 4: 연속적이지 않은 행의 수: 0
Group 5: 연속적이지 않은 행의 수: 1
Group 6: 연속적이지 않은 행의 수: 0
Group 7: 연속적이지 않은 행의 수: 0
Group 8: 연속적이지 않은 행의 수: 0
Group 9: 연속적이지 않은 행의 수: 0
Group 10: 연속적이지 않은 행의 수: 0
Group 11: 연속적이지 않은 행의 수: 0
Group 12: 연속적이지 않은 행의 수: 0
Group 13: 연속적이지 않은 행의 수: 0
Group 14: 연속적이지 않은 행의 수: 0
Group 15: 연속적이지 않은 행의 수: 0
Group 16: 연속적이지 않은 행의 수: 0
Group 17: 연속적이지 않은 행의 수: 0
Group 18: 연속적이지 않은 행의 수: 1
Group 19: 연속적이지 않은 행의 수: 0
Group 20: 연속적이지 않은 행의 수: 0
Group 21: 연속적이지 않은 행의 수: 0
Group 22: 연속적이지 않은 행의 수: 0
Group 23: 연속적이지 않은 행의 수: 0
Group 24: 연속적이지 않은 행의 수: 0
Group 25: 연속적이지 않은 행의 수: 0
Group 26: 연속적이지 않은 행의 수: 0
Group 27: 연속적이지 않은 행의 수: 0
Group 28: 연속적이지 않은 행의 수: 0
Group 29: 연속적이지 않은 행의 수: 0
Group 30: 연속적이지 않은 행의 수: 0
Group 31: 연속적이지 않은 행의 수: 0
Group 32: 연속적이지 않은 행의 수: 0
Group 33: 연속적이지 않은 행의 수: 0
Group 34: 연속적이지 않은 행의 수: 0
Group 35: 연속적이지 않은 행의 수: 0
Group 36: 연속적이지 않은 행의 수: 0
Group 37: 연속적이지 않은 행의 수: 1
Group 38: 

In [32]:
import pandas as pd

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8722.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 숫자부터 연속적으로 1이 나왔던 마지막 숫자까지를 출력합니다.
group_data = []
for i, group in enumerate(groups):
    start_count = None
    end_count = None
    consecutive_count = 0
    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if start_count is None:
                start_count = row['count']
            end_count = row['count']
            consecutive_count += 1
        elif start_count is not None:
            group_data.append({
                '그룹이름': f'Group {i+1}',
                '처음_passorfail_1': start_count,
                '연속적으로_1이_나왔던_마지막_숫자': end_count,
                '연속적인_1의_개수': consecutive_count
            })
            start_count = None
            end_count = None
            consecutive_count = 0

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(group_data)
result_df.head()

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,Group 2,1,16,16
1,Group 3,1,5,5
2,Group 3,8,9,2
3,Group 3,78,80,3
4,Group 3,157,157,1


In [33]:
# 각 그룹별 첫 번째 행을 뽑아서 새로운 데이터프레임으로 만듭니다.
first_rows_df = result_df.groupby('그룹이름').first().reset_index()

# 그룹 이름을 순서대로 정렬합니다.
first_rows_df['그룹이름'] = first_rows_df['그룹이름'].str.extract('(\d+)').astype(int)
first_rows_df = first_rows_df.sort_values('그룹이름').reset_index(drop=True)

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

# 결과를 출력합니다.
first_rows_df

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,2,1,16,16
1,3,1,5,5
2,4,1,5,5
3,5,1,4,4
4,6,1,5,5
5,7,1,5,5
6,8,1,5,5
7,9,1,13,13
8,10,1,5,5
9,11,1,4,4


In [34]:
# first_rows_df의 '처음으로_연속으로_1이_나온_개수' 열의 모든 값을 더합니다.
df_8722_first_drs = first_rows_df['연속적인_1의_개수'].sum()

# df_8722에서 passorfail이 1인 전체 데이터 개수
passorfail_1_count_8722 = len(df_8722[df_8722['passorfail'] == 1])

# df_8722의 전체 데이터 개수
total_data_count_8722 = len(df_8722)

# passorfail이 1인 전체 데이터 개수 / df_8722의 전체 데이터 개수 * 100 계산
defect_rate_8722 = (passorfail_1_count_8722 / total_data_count_8722) * 100

# df_8722_first_drs / df_8722에서 passorfail이 1인 전체 데이터 개수 * 100 계산
defect_ratio_8722 = (df_8722_first_drs / passorfail_1_count_8722) * 100

# 결과 출력
print("8722 passorfail이 1인 전체 데이터 개수:", passorfail_1_count_8722)
print("df_8722의 전체 데이터 개수:", total_data_count_8722)
print("8722 불량률:", defect_rate_8722)
print("8722 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율:", defect_ratio_8722)


8722 passorfail이 1인 전체 데이터 개수: 1088
df_8722의 전체 데이터 개수: 20048
8722 불량률: 5.426975259377494
8722 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율: 42.830882352941174


#### 8573


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92015 entries, 0 to 92014
Data columns (total 31 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   line                          92015 non-null  object 
 1   name                          92015 non-null  object 
 2   mold_name                     92015 non-null  object 
 3   time                          92015 non-null  object 
 4   date                          92015 non-null  object 
 5   count                         92015 non-null  int64  
 6   working                       92014 non-null  object 
 7   emergency_stop                92014 non-null  object 
 8   molten_temp                   89754 non-null  float64
 9   facility_operation_cycleTime  92015 non-null  int64  
 10  production_cycletime          92015 non-null  int64  
 11  low_section_speed             92014 non-null  float64
 12  high_section_speed            92014 non-null  float64
 13  m

In [36]:
# 그룹을 저장할 리스트 초기화
groups_8573 = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8573.iterrows():
    if row['count'] == 1 and current_group:
        groups_8573.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups_8573.append(pd.DataFrame(current_group))

# 각 그룹별로 count가 연속적인지 확인하는 코드
non_continuous_groups_count_8573 = 0
for i, group in enumerate(groups_8573):
    non_continuous_count = 0
    previous_count = None
    
    for index, row in group.iterrows():
        if previous_count is not None and row['count'] != previous_count + 1:
            non_continuous_count += 1
        previous_count = row['count']
    
    print(f"Group {i+1}: 연속적이지 않은 행의 수: {non_continuous_count}")
    
    # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
    if non_continuous_count != 0:
        non_continuous_groups_count_8573 += 1

# 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 출력합니다.
print(f"값이 0이 아닌 그룹의 수: {non_continuous_groups_count_8573}")


Group 1: 연속적이지 않은 행의 수: 1
Group 2: 연속적이지 않은 행의 수: 0
Group 3: 연속적이지 않은 행의 수: 0
Group 4: 연속적이지 않은 행의 수: 4
Group 5: 연속적이지 않은 행의 수: 0
Group 6: 연속적이지 않은 행의 수: 0
Group 7: 연속적이지 않은 행의 수: 0
Group 8: 연속적이지 않은 행의 수: 0
Group 9: 연속적이지 않은 행의 수: 4
Group 10: 연속적이지 않은 행의 수: 0
Group 11: 연속적이지 않은 행의 수: 0
Group 12: 연속적이지 않은 행의 수: 508
Group 13: 연속적이지 않은 행의 수: 2
Group 14: 연속적이지 않은 행의 수: 0
Group 15: 연속적이지 않은 행의 수: 0
Group 16: 연속적이지 않은 행의 수: 0
Group 17: 연속적이지 않은 행의 수: 0
Group 18: 연속적이지 않은 행의 수: 0
Group 19: 연속적이지 않은 행의 수: 1
Group 20: 연속적이지 않은 행의 수: 0
Group 21: 연속적이지 않은 행의 수: 2
Group 22: 연속적이지 않은 행의 수: 0
Group 23: 연속적이지 않은 행의 수: 0
Group 24: 연속적이지 않은 행의 수: 0
Group 25: 연속적이지 않은 행의 수: 4
Group 26: 연속적이지 않은 행의 수: 0
Group 27: 연속적이지 않은 행의 수: 0
Group 28: 연속적이지 않은 행의 수: 0
Group 29: 연속적이지 않은 행의 수: 0
Group 30: 연속적이지 않은 행의 수: 0
Group 31: 연속적이지 않은 행의 수: 0
Group 32: 연속적이지 않은 행의 수: 2
Group 33: 연속적이지 않은 행의 수: 0
Group 34: 연속적이지 않은 행의 수: 0
값이 0이 아닌 그룹의 수: 9


In [37]:
import pandas as pd

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8573.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 숫자부터 연속적으로 1이 나왔던 마지막 숫자까지를 출력합니다.
group_data = []
for i, group in enumerate(groups):
    start_count = None
    end_count = None
    consecutive_count = 0
    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if start_count is None:
                start_count = row['count']
            end_count = row['count']
            consecutive_count += 1
        elif start_count is not None:
            group_data.append({
                '그룹이름': f'Group {i+1}',
                '처음_passorfail_1': start_count,
                '연속적으로_1이_나왔던_마지막_숫자': end_count,
                '연속적인_1의_개수': consecutive_count
            })
            start_count = None
            end_count = None
            consecutive_count = 0

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(group_data)
result_df.head()

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,Group 1,2,5,4
1,Group 2,1,5,5
2,Group 2,177,177,1
3,Group 2,190,191,2
4,Group 3,1,6,6


In [38]:
# 각 그룹별 첫 번째 행을 뽑아서 새로운 데이터프레임으로 만듭니다.
first_rows_df = result_df.groupby('그룹이름').first().reset_index()

# 그룹 이름을 순서대로 정렬합니다.
first_rows_df['그룹이름'] = first_rows_df['그룹이름'].str.extract('(\d+)').astype(int)
first_rows_df = first_rows_df.sort_values('그룹이름').reset_index(drop=True)

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

# 결과를 출력합니다.
first_rows_df

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,1,2,5,4
1,2,1,5,5
2,3,1,6,6
3,4,1,11,11
4,5,1,5,5
5,6,1,6,6
6,7,1,7,7
7,8,1,5,5
8,9,1,6,6
9,10,1,5,5


In [39]:
# first_rows_df의 '처음으로_연속으로_1이_나온_개수' 열의 모든 값을 더합니다.
df_8573_first_drs = first_rows_df['연속적인_1의_개수'].sum()

# df_8573에서 passorfail이 1인 전체 데이터 개수
passorfail_1_count_8573 = len(df_8573[df_8573['passorfail'] == 1])

# df_8573의 전체 데이터 개수
total_data_count_8573 = len(df_8573)

# passorfail이 1인 전체 데이터 개수 / df_8573의 전체 데이터 개수 * 100 계산
defect_rate_8573 = (passorfail_1_count_8573 / total_data_count_8573) * 100

# df_8573_first_drs / df_8573에서 passorfail이 1인 전체 데이터 개수 * 100 계산
defect_ratio_8573 = (df_8573_first_drs / passorfail_1_count_8573) * 100

# 결과 출력
print("8573 passorfail이 1인 전체 데이터 개수:", passorfail_1_count_8573)
print("df_8573의 전체 데이터 개수:", total_data_count_8573)
print("8573 불량률:", defect_rate_8573)
print("8573 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율:", defect_ratio_8573)

8573 passorfail이 1인 전체 데이터 개수: 420
df_8573의 전체 데이터 개수: 9596
8573 불량률: 4.376823676531888
8573 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율: 44.52380952380952


#### 8413


In [40]:
# 그룹을 저장할 리스트 초기화
groups_8413 = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8413.iterrows():
    if row['count'] == 1 and current_group:
        groups_8413.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups_8413.append(pd.DataFrame(current_group))

# 각 그룹별로 count가 연속적인지 확인하는 코드
non_continuous_groups_count_8413 = 0
for i, group in enumerate(groups_8413):
    non_continuous_count = 0
    previous_count = None
    
    for index, row in group.iterrows():
        if previous_count is not None and row['count'] != previous_count + 1:
            non_continuous_count += 1
        previous_count = row['count']
    
    print(f"Group {i+1}: 연속적이지 않은 행의 수: {non_continuous_count}")
    
    # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
    if non_continuous_count != 0:
        non_continuous_groups_count_8413 += 1

# 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 출력합니다.
print(f"값이 0이 아닌 그룹의 수: {non_continuous_groups_count_8413}")


Group 1: 연속적이지 않은 행의 수: 0
Group 2: 연속적이지 않은 행의 수: 0
Group 3: 연속적이지 않은 행의 수: 0
Group 4: 연속적이지 않은 행의 수: 0
Group 5: 연속적이지 않은 행의 수: 0
Group 6: 연속적이지 않은 행의 수: 3
Group 7: 연속적이지 않은 행의 수: 0
Group 8: 연속적이지 않은 행의 수: 0
Group 9: 연속적이지 않은 행의 수: 0
Group 10: 연속적이지 않은 행의 수: 0
Group 11: 연속적이지 않은 행의 수: 0
Group 12: 연속적이지 않은 행의 수: 0
Group 13: 연속적이지 않은 행의 수: 0
Group 14: 연속적이지 않은 행의 수: 4
Group 15: 연속적이지 않은 행의 수: 0
Group 16: 연속적이지 않은 행의 수: 0
Group 17: 연속적이지 않은 행의 수: 0
Group 18: 연속적이지 않은 행의 수: 1
Group 19: 연속적이지 않은 행의 수: 0
Group 20: 연속적이지 않은 행의 수: 0
값이 0이 아닌 그룹의 수: 3


In [41]:
import pandas as pd

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8413.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 숫자부터 연속적으로 1이 나왔던 마지막 숫자까지를 출력합니다.
group_data = []
for i, group in enumerate(groups):
    start_count = None
    end_count = None
    consecutive_count = 0
    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if start_count is None:
                start_count = row['count']
            end_count = row['count']
            consecutive_count += 1
        elif start_count is not None:
            group_data.append({
                '그룹이름': f'Group {i+1}',
                '처음_passorfail_1': start_count,
                '연속적으로_1이_나왔던_마지막_숫자': end_count,
                '연속적인_1의_개수': consecutive_count
            })
            start_count = None
            end_count = None
            consecutive_count = 0

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(group_data)
result_df.head()


,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,Group 1,1,5,5
1,Group 1,78,78,1
2,Group 1,114,114,1
3,Group 1,178,178,1
4,Group 2,1,5,5


In [42]:
# 각 그룹별 첫 번째 행을 뽑아서 새로운 데이터프레임으로 만듭니다.
first_rows_df = result_df.groupby('그룹이름').first().reset_index()

# 그룹 이름을 순서대로 정렬합니다.
first_rows_df['그룹이름'] = first_rows_df['그룹이름'].str.extract('(\d+)').astype(int)
first_rows_df = first_rows_df.sort_values('그룹이름').reset_index(drop=True)

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

# 결과를 출력합니다.
first_rows_df

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,1,1,5,5
1,2,1,5,5
2,3,1,5,5
3,4,1,6,6
4,5,1,10,10
5,6,1,2,2
6,7,1,5,5
7,8,1,6,6
8,9,1,5,5
9,10,1,5,5


In [43]:
# first_rows_df의 '처음으로_연속으로_1이_나온_개수' 열의 모든 값을 더합니다.
df_8413_first_drs = first_rows_df['연속적인_1의_개수'].sum()

# df_8413에서 passorfail이 1인 전체 데이터 개수
passorfail_1_count_8413 = len(df_8413[df_8413['passorfail'] == 1])

# df_8413의 전체 데이터 개수
total_data_count_8413 = len(df_8413)

# passorfail이 1인 전체 데이터 개수 / df_8413의 전체 데이터 개수 * 100 계산
defect_rate_8413 = (passorfail_1_count_8413 / total_data_count_8413) * 100

# df_8413_first_drs / df_8413에서 passorfail이 1인 전체 데이터 개수 * 100 계산
defect_ratio_8413 = (df_8413_first_drs / passorfail_1_count_8413) * 100

# 결과 출력
print("8413 passorfail이 1인 전체 데이터 개수:", passorfail_1_count_8413)
print("df_8413의 전체 데이터 개수:", total_data_count_8413)
print("8413 불량률:", defect_rate_8413)
print("8413 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율:", defect_ratio_8413)


8413 passorfail이 1인 전체 데이터 개수: 226
df_8413의 전체 데이터 개수: 6034
8413 불량률: 3.7454424925422605
8413 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율: 57.07964601769911


#### 8600

In [44]:
# 그룹을 저장할 리스트 초기화
groups_8600 = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8600.iterrows():
    if row['count'] == 1 and current_group:
        groups_8600.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups_8600.append(pd.DataFrame(current_group))

# 각 그룹별로 count가 연속적인지 확인하는 코드
non_continuous_groups_count_8600 = 0
for i, group in enumerate(groups_8600):
    non_continuous_count = 0
    previous_count = None
    
    for index, row in group.iterrows():
        if previous_count is not None and row['count'] != previous_count + 1:
            non_continuous_count += 1
        previous_count = row['count']
    
    print(f"Group {i+1}: 연속적이지 않은 행의 수: {non_continuous_count}")
    
    # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
    if non_continuous_count != 0:
        non_continuous_groups_count_8600 += 1

# 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 출력합니다.
print(f"값이 0이 아닌 그룹의 수: {non_continuous_groups_count_8600}")


Group 1: 연속적이지 않은 행의 수: 0
Group 2: 연속적이지 않은 행의 수: 0
Group 3: 연속적이지 않은 행의 수: 0
Group 4: 연속적이지 않은 행의 수: 0
Group 5: 연속적이지 않은 행의 수: 0
Group 6: 연속적이지 않은 행의 수: 0
Group 7: 연속적이지 않은 행의 수: 0
Group 8: 연속적이지 않은 행의 수: 0
Group 9: 연속적이지 않은 행의 수: 0
Group 10: 연속적이지 않은 행의 수: 0
Group 11: 연속적이지 않은 행의 수: 0
Group 12: 연속적이지 않은 행의 수: 0
값이 0이 아닌 그룹의 수: 0


In [45]:
import pandas as pd

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8600.iterrows():
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 숫자부터 연속적으로 1이 나왔던 마지막 숫자까지를 출력합니다.
group_data = []
for i, group in enumerate(groups):
    start_count = None
    end_count = None
    consecutive_count = 0
    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if start_count is None:
                start_count = row['count']
            end_count = row['count']
            consecutive_count += 1
        elif start_count is not None:
            group_data.append({
                '그룹이름': f'Group {i+1}',
                '처음_passorfail_1': start_count,
                '연속적으로_1이_나왔던_마지막_숫자': end_count,
                '연속적인_1의_개수': consecutive_count
            })
            start_count = None
            end_count = None
            consecutive_count = 0

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(group_data)
result_df.head()


,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,Group 2,1,4,4
1,Group 2,34,34,1
2,Group 2,36,36,1
3,Group 2,43,44,2
4,Group 2,58,59,2


In [46]:
# 각 그룹별 첫 번째 행을 뽑아서 새로운 데이터프레임으로 만듭니다.
first_rows_df = result_df.groupby('그룹이름').first().reset_index()

# 그룹 이름을 순서대로 정렬합니다.
first_rows_df['그룹이름'] = first_rows_df['그룹이름'].str.extract('(\d+)').astype(int)
first_rows_df = first_rows_df.sort_values('그룹이름').reset_index(drop=True)

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

# 결과를 출력합니다.
first_rows_df

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,2,1,4,4
1,3,1,3,3
2,4,1,4,4
3,5,43,44,2
4,6,1,4,4
5,7,19,19,1
6,8,1,5,5
7,9,1,1,1
8,10,1,5,5
9,11,1,8,8


In [47]:
# first_rows_df의 '처음으로_연속으로_1이_나온_개수' 열의 모든 값을 더합니다.
df_8600_first_drs = first_rows_df['연속적인_1의_개수'].sum()

# df_8600에서 passorfail이 1인 전체 데이터 개수
passorfail_1_count_8600 = len(df_8600[df_8600['passorfail'] == 1])

# df_8600의 전체 데이터 개수
total_data_count_8600 = len(df_8600)

# passorfail이 1인 전체 데이터 개수 / df_8600의 전체 데이터 개수 * 100 계산
defect_rate_8600 = (passorfail_1_count_8600 / total_data_count_8600) * 100

# df_8600_first_drs / df_8600에서 passorfail이 1인 전체 데이터 개수 * 100 계산
defect_ratio_8600 = (df_8600_first_drs / passorfail_1_count_8600) * 100

# 결과 출력
print("8600 passorfail이 1인 전체 데이터 개수:", passorfail_1_count_8600)
print("df_8600의 전체 데이터 개수:", total_data_count_8600)
print("8600 불량률:", defect_rate_8600)
print("8600 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율:", defect_ratio_8600)


8600 passorfail이 1인 전체 데이터 개수: 150
df_8600의 전체 데이터 개수: 2960
8600 불량률: 5.0675675675675675
8600 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율: 26.666666666666668


#### 8576

In [48]:
# 그룹을 저장할 리스트 초기화
groups_8576 = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8576.iterrows():
    if row['count'] == 1 and current_group:
        groups_8576.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups_8576.append(pd.DataFrame(current_group))

# 각 그룹별로 count가 연속적인지 확인하는 코드
non_continuous_groups_count_8576 = 0
for i, group in enumerate(groups_8576):
    non_continuous_count = 0
    previous_count = None
    
    for index, row in group.iterrows():
        if previous_count is not None and row['count'] != previous_count + 1:
            non_continuous_count += 1
        previous_count = row['count']
    
    print(f"Group {i+1}: 연속적이지 않은 행의 수: {non_continuous_count}")
    
    # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
    if non_continuous_count != 0:
        non_continuous_groups_count_8576 += 1

# 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 출력합니다.
print(f"값이 0이 아닌 그룹의 수: {non_continuous_groups_count_8576}")


Group 1: 연속적이지 않은 행의 수: 0
Group 2: 연속적이지 않은 행의 수: 0
Group 3: 연속적이지 않은 행의 수: 2
Group 4: 연속적이지 않은 행의 수: 0
Group 5: 연속적이지 않은 행의 수: 0
값이 0이 아닌 그룹의 수: 1


In [49]:
import pandas as pd

# 그룹을 저장할 리스트 초기화
groups = []
current_group = []

# 각 행을 순회하면서 그룹을 만듭니다.
for index, row in df_8576.iterrows():  # df_8600 -> df_8576로 수정
    if row['count'] == 1 and current_group:
        groups.append(pd.DataFrame(current_group))
        current_group = [row]
    else:
        current_group.append(row)

# 마지막 그룹 추가 (남아있는 경우)
if current_group:
    groups.append(pd.DataFrame(current_group))

# 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 숫자부터 연속적으로 1이 나왔던 마지막 숫자까지를 출력합니다.
group_data = []
for i, group in enumerate(groups):
    start_count = None
    end_count = None
    consecutive_count = 0
    for index, row in group.iterrows():
        if row['passorfail'] == 1:
            if start_count is None:
                start_count = row['count']
            end_count = row['count']
            consecutive_count += 1
        elif start_count is not None:
            group_data.append({
                '그룹이름': f'Group {i+1}',
                '처음_passorfail_1': start_count,
                '연속적으로_1이_나왔던_마지막_숫자': end_count,
                '연속적인_1의_개수': consecutive_count
            })
            start_count = None
            end_count = None
            consecutive_count = 0

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(group_data)
result_df.head()


,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,Group 1,1,7,7
1,Group 1,87,87,1
2,Group 2,1,9,9
3,Group 2,42,43,2
4,Group 3,1,5,5


In [50]:
# 각 그룹별 첫 번째 행을 뽑아서 새로운 데이터프레임으로 만듭니다.
first_rows_df = result_df.groupby('그룹이름').first().reset_index()

# 그룹 이름을 순서대로 정렬합니다.
first_rows_df['그룹이름'] = first_rows_df['그룹이름'].str.extract('(\d+)').astype(int)
first_rows_df = first_rows_df.sort_values('그룹이름').reset_index(drop=True)

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

# 결과를 출력합니다.
first_rows_df

,그룹이름,처음_passorfail_1,연속적으로_1이_나왔던_마지막_숫자,연속적인_1의_개수
0,1,1,7,7
1,2,1,9,9
2,3,1,5,5
3,4,1,6,6
4,5,1,5,5


In [51]:
# first_rows_df의 '처음으로_연속으로_1이_나온_개수' 열의 모든 값을 더합니다.
df_8576_first_drs = first_rows_df['연속적인_1의_개수'].sum()

# df_8576에서 passorfail이 1인 전체 데이터 개수
passorfail_1_count_8576 = len(df_8576[df_8576['passorfail'] == 1])

# df_8576의 전체 데이터 개수
total_data_count_8576 = len(df_8576)

# passorfail이 1인 전체 데이터 개수 / df_8576의 전체 데이터 개수 * 100 계산
defect_rate_8576 = (passorfail_1_count_8576 / total_data_count_8576) * 100

# df_8576_first_drs / df_8576에서 passorfail이 1인 전체 데이터 개수 * 100 계산
defect_ratio_8576 = (df_8576_first_drs / passorfail_1_count_8576) * 100

# 결과 출력
print("8576 passorfail이 1인 전체 데이터 개수:", passorfail_1_count_8576)
print("df_8576의 전체 데이터 개수:", total_data_count_8576)
print("8576 불량률:", defect_rate_8576)
print("8576 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율:", defect_ratio_8576)


8576 passorfail이 1인 전체 데이터 개수: 40
df_8576의 전체 데이터 개수: 793
8576 불량률: 5.044136191677175
8576 처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율: 80.0


### 💡전체 mold_code에 대해서 t-test 결과

In [52]:
import pandas as pd
from scipy.stats import ttest_ind

# 그룹 데이터프레임 리스트
dfs = [df_8917, df_8412, df_8722, df_8573, df_8413, df_8600, df_8576]

# t-test를 수행할 열들
columns_to_test = ['molten_temp', 'facility_operation_cycleTime', 'production_cycletime', 'low_section_speed',
                   'high_section_speed', 'molten_volume', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
                   'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3',
                   'sleeve_temperature', 'physical_strength', 'Coolant_temperature', 'EMS_operation_time']

# 그룹별로 t-test 수행
for df in dfs:
    # 그룹 이름 추출
    group_name = df['mold_code'].iloc[0]

    # 그룹을 저장할 리스트 초기화
    groups = []
    current_group = []

    # 각 행을 순회하면서 그룹을 만듭니다.
    for index, row in df.iterrows():
        if row['count'] == 1 and current_group:
            groups.append(pd.DataFrame(current_group))
            current_group = [row]
        else:
            current_group.append(row)

    # 마지막 그룹 추가 (남아있는 경우)
    if current_group:
        groups.append(pd.DataFrame(current_group))

    # 연속적인 1과 0 데이터를 저장할 리스트 초기화
    group_1_all = []
    group_0_all = []

    for i, group in enumerate(groups):
        start_count_1 = None
        end_count_1 = None
        start_count_0 = None
        end_count_0 = None
        consecutive_count_1 = 0
        consecutive_count_0 = 0
        found_first_1 = False
        found_first_0 = False

        for index, row in group.iterrows():
            if row['passorfail'] == 1:
                if not found_first_1:
                    start_count_1 = row['count']
                    found_first_1 = True
                end_count_1 = row['count']
                consecutive_count_1 += 1
            elif found_first_1 and not found_first_0:
                start_count_0 = row['count']
                found_first_0 = True
                consecutive_count_0 += 1
                end_count_0 = row['count']
            elif found_first_0 and row['passorfail'] == 0:
                end_count_0 = row['count']
                consecutive_count_0 += 1

        # 그룹 나누기
        group_1 = group[(group['count'] >= start_count_1) & (group['count'] <= end_count_1)]
        group_0 = group[(group['count'] >= start_count_0) & (group['count'] <= end_count_0)]

        group_1_all.append(group_1)
        group_0_all.append(group_0)

    # 그룹 데이터프레임 합치기
    group_1_combined = pd.concat(group_1_all, ignore_index=True)
    group_0_combined = pd.concat(group_0_all, ignore_index=True)

    # t-test 수행
    significant_columns = []

    for col in columns_to_test:
        if not group_1_combined[col].dropna().empty and not group_0_combined[col].dropna().empty:
            t_stat, p_value = ttest_ind(group_1_combined[col].dropna(), group_0_combined[col].dropna(), equal_var=False)
            if p_value < 0.05:
                significant_columns.append((col, t_stat, p_value))

    # 결과 출력
    print(f"df_{group_name} Significant Columns:")
    for col, t_stat, p_value in significant_columns:
        print(f"{col}: t-statistic = {t_stat}, p-value = {p_value}")

    print("\n")


df_8917 Significant Columns:
molten_temp: t-statistic = 4.474951263377227, p-value = 7.663658518850572e-06
production_cycletime: t-statistic = -2.4889688993243038, p-value = 0.012815160896404299
low_section_speed: t-statistic = -8.927176137806928, p-value = 4.586357521195487e-19
cast_pressure: t-statistic = -20.906880480459712, p-value = 2.5311646156809974e-96
biscuit_thickness: t-statistic = 3.0789415205740998, p-value = 0.002079340362553751
upper_mold_temp1: t-statistic = 4.343652947586796, p-value = 1.4045449408986237e-05
upper_mold_temp2: t-statistic = -12.171278323950038, p-value = 5.119508425122805e-34
lower_mold_temp1: t-statistic = -15.709039972224923, p-value = 1.9249605010170332e-55
lower_mold_temp2: t-statistic = -2.0727715419957464, p-value = 0.03819978256961022
sleeve_temperature: t-statistic = 2.000518908286667, p-value = 0.04545042921668579
physical_strength: t-statistic = -7.3653636823257616, p-value = 1.8162939012371921e-13
Coolant_temperature: t-statistic = -21.060876

### 🧑‍🏫 인사이트 도출

#### 8917

In [53]:
import pandas as pd
import statsmodels.api as sm

# 데이터로드
df = df_8917

# 독립 변수와 종속 변수 설정
X = df[['molten_temp', 'production_cycletime', 'low_section_speed', 'cast_pressure',
        'biscuit_thickness', 'upper_mold_temp1', 'upper_mold_temp2', 'lower_mold_temp1',
        'lower_mold_temp2', 'sleeve_temperature', 'physical_strength', 'Coolant_temperature']]
y = df['passorfail']  # 예측하려는 타겟 변수 (예: 0 또는 1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀 모델 피팅
model = sm.Logit(y, X)
result = model.fit()

# 결과 요약
print(result.summary())

# 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
significant_vars = result.pvalues[result.pvalues < 0.05].index
print("Significant Variables: ", significant_vars)

# 중요한 변수들의 평균 및 분산 계산
important_vars = significant_vars.drop('const')
important_vars_data = df[important_vars]
means = important_vars_data.mean()
stds = important_vars_data.std()

# 최적 범위 추정 (예: 평균 ± 1 표준편차)
optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in important_vars}
print("Optimal Ranges: ", optimal_ranges)


Optimization terminated successfully.
         Current function value: 0.046376
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                31112
Model:                          Logit   Df Residuals:                    31099
Method:                           MLE   Df Model:                           12
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.7400
Time:                        10:27:39   Log-Likelihood:                -1442.8
converged:                       True   LL-Null:                       -5549.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   64.7709      8.730      7.419      0.000      47.660      81.88

#### 8412

In [54]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터로드
df = df_8412

# 독립 변수와 종속 변수 설정
X = df[['high_section_speed', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1', 'upper_mold_temp2',
        'lower_mold_temp2', 'sleeve_temperature', 'EMS_operation_time']]
y = df['passorfail']  # 예측하려는 타겟 변수 (예: 0 또는 1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀 모델 피팅
model = sm.Logit(y, X)
result = model.fit()

# 결과 요약
print(result.summary())

# 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
significant_vars = result.pvalues[result.pvalues < 0.05].index
print("Significant Variables: ", significant_vars)

# 중요한 변수들의 평균 및 분산 계산
important_vars = significant_vars.drop('const')
important_vars_data = df[important_vars]
means = important_vars_data.mean()
stds = important_vars_data.std()

# 최적 범위 추정 (예: 평균 ± 1 표준편차)
optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in important_vars}
print("Optimal Ranges: ", optimal_ranges)

Optimization terminated successfully.
         Current function value: 0.028855
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                21471
Model:                          Logit   Df Residuals:                    21462
Method:                           MLE   Df Model:                            8
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.8083
Time:                        10:27:39   Log-Likelihood:                -619.55
converged:                       True   LL-Null:                       -3232.6
Covariance Type:            nonrobust   LLR p-value:                     0.000
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 21.2469      1.152     18.440      0.000      18.989      23.505
high

#### 8722

In [55]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터로드
df = df_8722

# 독립 변수와 종속 변수 설정
X = df[['production_cycletime', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1', 'upper_mold_temp2',
        'lower_mold_temp1', 'lower_mold_temp2', 'sleeve_temperature', 'physical_strength', 'Coolant_temperature']]
y = df['passorfail']  # 예측하려는 타겟 변수 (예: 0 또는 1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀 모델 피팅
model = sm.Logit(y, X)
result = model.fit()

# 결과 요약
print(result.summary())

# 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
significant_vars = result.pvalues[result.pvalues < 0.05].index
print("Significant Variables: ", significant_vars)

# 중요한 변수들의 평균 및 분산 계산
important_vars = significant_vars.drop('const')
important_vars_data = df[important_vars]
means = important_vars_data.mean()
stds = important_vars_data.std()

# 최적 범위 추정 (예: 평균 ± 1 표준편차)
optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in important_vars}
print("Optimal Ranges: ", optimal_ranges)

Optimization terminated successfully.
         Current function value: 0.103629
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                20048
Model:                          Logit   Df Residuals:                    20037
Method:                           MLE   Df Model:                           10
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.5086
Time:                        10:27:39   Log-Likelihood:                -2077.5
converged:                       True   LL-Null:                       -4228.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   38.3596      2.864     13.396      0.000      32.747      43.97

#### 8573

In [56]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터로드
df = df_8573

# 독립 변수와 종속 변수 설정
X = df[['low_section_speed', 'cast_pressure', 'upper_mold_temp1', 'upper_mold_temp2',
        'lower_mold_temp1', 'lower_mold_temp2', 'sleeve_temperature', 'Coolant_temperature']]
y = df['passorfail']  # 예측하려는 타겟 변수 (예: 0 또는 1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀 모델 피팅
model = sm.Logit(y, X)
result = model.fit()

# 결과 요약
print(result.summary())

# 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
significant_vars = result.pvalues[result.pvalues < 0.05].index
print("Significant Variables: ", significant_vars)

# 중요한 변수들의 평균 및 분산 계산
important_vars = significant_vars.drop('const')
important_vars_data = df[important_vars]
means = important_vars_data.mean()
stds = important_vars_data.std()

# 최적 범위 추정 (예: 평균 ± 1 표준편차)
optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in important_vars}
print("Optimal Ranges: ", optimal_ranges)

Optimization terminated successfully.
         Current function value: 0.020220
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                 9596
Model:                          Logit   Df Residuals:                     9587
Method:                           MLE   Df Model:                            8
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.8875
Time:                        10:27:39   Log-Likelihood:                -194.04
converged:                       True   LL-Null:                       -1724.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  26.1178      3.436      7.601      0.000      19.383      32.853
l

#### 8413

In [57]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터로드
df = df_8413

# 독립 변수와 종속 변수 설정
X = df[['molten_temp', 'production_cycletime', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp2',
        'lower_mold_temp1', 'lower_mold_temp2', 'sleeve_temperature', 'physical_strength', 'Coolant_temperature']]
y = df['passorfail']  # 예측하려는 타겟 변수 (예: 0 또는 1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀 모델 피팅
model = sm.Logit(y, X)
result = model.fit()

# 결과 요약
print(result.summary())

# 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
significant_vars = result.pvalues[result.pvalues < 0.05].index
print("Significant Variables: ", significant_vars)

# 중요한 변수들의 평균 및 분산 계산
important_vars = significant_vars.drop('const')
important_vars_data = df[important_vars]
means = important_vars_data.mean()
stds = important_vars_data.std()

# 최적 범위 추정 (예: 평균 ± 1 표준편차)
optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in important_vars}
print("Optimal Ranges: ", optimal_ranges)

Optimization terminated successfully.
         Current function value: 0.044036
         Iterations 17
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                 6034
Model:                          Logit   Df Residuals:                     6023
Method:                           MLE   Df Model:                           10
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.7244
Time:                        10:27:39   Log-Likelihood:                -265.72
converged:                       True   LL-Null:                       -964.04
Covariance Type:            nonrobust   LLR p-value:                5.250e-294
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  220.6516     52.925      4.169      0.000     116.920     324.38

#### 8600

In [58]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터로드
df = df_8600

# 독립 변수와 종속 변수 설정
X = df[['cast_pressure', 'biscuit_thickness', 'lower_mold_temp2', 'sleeve_temperature', 'physical_strength', 'Coolant_temperature']]
y = df['passorfail']  # 예측하려는 타겟 변수 (예: 0 또는 1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀 모델 피팅
model = sm.Logit(y, X)
result = model.fit()

# 결과 요약
print(result.summary())

# 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
significant_vars = result.pvalues[result.pvalues < 0.05].index
print("Significant Variables: ", significant_vars)

# 중요한 변수들의 평균 및 분산 계산
important_vars = significant_vars.drop('const')
important_vars_data = df[important_vars]
means = important_vars_data.mean()
stds = important_vars_data.std()

# 최적 범위 추정 (예: 평균 ± 1 표준편차)
optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in important_vars}
print("Optimal Ranges: ", optimal_ranges)

Optimization terminated successfully.
         Current function value: 0.070864
         Iterations 12
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                 2960
Model:                          Logit   Df Residuals:                     2953
Method:                           MLE   Df Model:                            6
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.6466
Time:                        10:27:39   Log-Likelihood:                -209.76
converged:                       True   LL-Null:                       -593.48
Covariance Type:            nonrobust   LLR p-value:                1.662e-162
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 315.0821     73.323      4.297      0.000     171.371     458.793
c

#### 8576

In [59]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 데이터로드
df = df_8576

# 독립 변수와 종속 변수 설정
X = df[['molten_temp', 'low_section_speed', 'cast_pressure','upper_mold_temp2',
        'lower_mold_temp2', 'sleeve_temperature', 'physical_strength', 'EMS_operation_time']]
y = df['passorfail']  # 예측하려는 타겟 변수 (예: 0 또는 1)

# 상수항 추가
X = sm.add_constant(X)

# 로지스틱 회귀 모델 피팅
model = sm.Logit(y, X)
result = model.fit()

# 결과 요약
print(result.summary())

# 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
significant_vars = result.pvalues[result.pvalues < 0.05].index
print("Significant Variables: ", significant_vars)

# 중요한 변수들의 평균 및 분산 계산
important_vars_data = df[important_vars]
means = important_vars_data.mean()
stds = important_vars_data.std()

# 최적 범위 추정 (예: 평균 ± 1 표준편차)
optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in important_vars}
print("Optimal Ranges: ", optimal_ranges)

Optimization terminated successfully.
         Current function value: 0.032015
         Iterations 16
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                  793
Model:                          Logit   Df Residuals:                      784
Method:                           MLE   Df Model:                            8
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.8398
Time:                        10:27:39   Log-Likelihood:                -25.388
converged:                       True   LL-Null:                       -158.45
Covariance Type:            nonrobust   LLR p-value:                 6.532e-53
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 58.1321    339.252      0.171      0.864    -606.790     723.054
molt

### mold_code 별 count 연속성

In [60]:
# mold_code별 데이터를 저장할 리스트 초기화
mold_codes = [8917, 8412, 8722, 8573, 8413, 8600, 8576]
data = []

for code in mold_codes:
    df_code = data_CK[data_CK['mold_code'] == code]
    
    # 그룹을 저장할 리스트 초기화
    groups = []
    current_group = []
    
    # 각 행을 순회하면서 그룹을 만듭니다.
    for index, row in df_code.iterrows():
        if row['count'] == 1 and current_group:
            groups.append(pd.DataFrame(current_group))
            current_group = [row]
        else:
            current_group.append(row)
    
    # 마지막 그룹 추가 (남아있는 경우)
    if current_group:
        groups.append(pd.DataFrame(current_group))
    
    # 각 그룹별로 count가 연속적인지 확인하는 코드
    non_continuous_groups_count = 0
    total_non_continuous_rows = 0
    
    for i, group in enumerate(groups):
        non_continuous_count = 0
        previous_count = None
        
        for index, row in group.iterrows():
            if previous_count is not None and row['count'] != previous_count + 1:
                non_continuous_count += 1
            previous_count = row['count']
        
        # 연속적이지 않은 행의 수가 0이 아닌 그룹의 수를 셉니다.
        if non_continuous_count != 0:
            non_continuous_groups_count += 1
            total_non_continuous_rows += non_continuous_count
    
    # 결과를 리스트에 추가합니다.
    data.append({
        'mold_code': code,
        'non_continuous_groups': non_continuous_groups_count,
        'total_groups': len(groups),
        'non_continuous_rows': total_non_continuous_rows,
        'total_rows': len(df_code),
        'non_continuous_rate' : total_non_continuous_rows / len(df_code) * 100
    })

# 결과를 데이터프레임으로 변환합니다.
result_df = pd.DataFrame(data)
result_df



,mold_code,non_continuous_groups,total_groups,non_continuous_rows,total_rows,non_continuous_rate
0,8917,10,107,1119,31112,3.596683
1,8412,13,77,24,21471,0.111779
2,8722,10,75,15,20048,0.074820
3,8573,9,34,528,9596,5.502293
4,8413,3,20,8,6034,0.132582
5,8600,0,12,0,2960,0.000000
6,8576,1,5,2,793,0.252207


### 불량률 / 처음 연속적이었던 count의 불량률

In [61]:
import pandas as pd

# 각 데이터에 대한 결과를 리스트에 저장합니다.
data = [
    {'데이터프레임이름': '8917', '불량률': defect_rate, '처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율': defect_ratio},
    {'데이터프레임이름': '8412', '불량률': defect_rate_8412, '처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율': defect_ratio_8412},
    {'데이터프레임이름': '8722', '불량률': defect_rate_8722, '처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율': defect_ratio_8722},
    {'데이터프레임이름': '8573', '불량률': defect_rate_8573, '처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율': defect_ratio_8573},
    {'데이터프레임이름': '8413', '불량률': defect_rate_8413, '처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율': defect_ratio_8413},
    {'데이터프레임이름': '8600', '불량률': defect_rate_8600, '처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율': defect_ratio_8600},
    {'데이터프레임이름': '8576', '불량률': defect_rate_8576, '처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율': defect_ratio_8576}
]

# 데이터를 데이터프레임으로 변환합니다.
overview_df = pd.DataFrame(data)

# 결과 출력
overview_df


,데이터프레임이름,불량률,처음 그룹별로 연속으로 1이 나온 첫 개수 / passorfail이 1인 전체 데이터 개수 비율
0,8917,4.332733,45.326409
1,8412,3.465139,53.360215
2,8722,5.426975,42.830882
3,8573,4.376824,44.523810
4,8413,3.745442,57.079646
5,8600,5.067568,26.666667
6,8576,5.044136,80.000000


### mold_code 별 EDA

In [62]:
import pandas as pd

# 출력 옵션 설정
pd.set_option('display.max_colwidth', None)

# 각 mold_code별 고윳값이 1개인 열을 찾고 저장할 리스트 초기화
unique_columns_data = []

for code in mold_codes:
    df_code = data_CK[data_CK['mold_code'] == code]
    
    # 각 열의 고윳값 수를 계산합니다. dropna=False를 설정하여 NaN을 무시하지 않고 카운트합니다.
    unique_value_counts = df_code.apply(lambda x: x.nunique(dropna=False))
    
    # 고윳값이 1개인 열들을 필터링합니다. (NaN 값이 포함된 열도 고윳값 1개로 취급됩니다.)
    single_unique_columns = unique_value_counts[unique_value_counts == 1].index.tolist()
    
    # 결과를 리스트에 추가합니다.
    unique_columns_data.append({
        'mold_code': code,
        'single_unique_columns': single_unique_columns,
        'column_count': len(single_unique_columns)
    })

# 결과를 데이터프레임으로 변환합니다.
unique_columns_df = pd.DataFrame(unique_columns_data)
unique_columns_df


,mold_code,single_unique_columns,column_count
0,8917,"[upper_mold_temp3, lower_mold_temp3, mold_code]",3
1,8412,[mold_code],1
2,8722,"[upper_mold_temp3, EMS_operation_time, mold_code, heating_furnace]",4
3,8573,"[molten_volume, lower_mold_temp3, EMS_operation_time, tryshot_signal, mold_code]",5
4,8413,"[upper_mold_temp3, lower_mold_temp3, EMS_operation_time, mold_code, heating_furnace]",5
5,8600,"[working, upper_mold_temp3, lower_mold_temp3, EMS_operation_time, tryshot_signal, mold_code, heating_furnace]",7
6,8576,"[molten_volume, upper_mold_temp3, lower_mold_temp3, mold_code]",4


## 🫨 t-test의 한계를 느끼고 다시 다중회귀 분석
장점:

간단하고 직관적입니다.
각 변수에 대해 개별적으로 유의미한 차이가 있는지를 확인할 수 있습니다.  
  
-----

단점:

각 변수 간의 상호작용을 고려하지 않습니다.
다중 변수 상황에서는 한 변수의 효과가 다른 변수의 영향을 받을 수 있는 경우를 고려하지 않습니다.

In [72]:
import pandas as pd
from scipy.stats import ttest_ind
import statsmodels.api as sm

# 그룹 데이터프레임 리스트
dfs = [df_8917, df_8412, df_8722, df_8573, df_8413, df_8600, df_8576]

# t-test를 수행할 열들
columns_to_test = ['molten_temp', 'facility_operation_cycleTime', 'production_cycletime', 'low_section_speed',
                   'high_section_speed', 'molten_volume', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
                   'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3',
                   'sleeve_temperature', 'physical_strength', 'Coolant_temperature', 'EMS_operation_time']

# 그룹별로 작업 수행
for df in dfs:
    # 그룹 이름 추출
    group_name = df['mold_code'].iloc[0]

    # 그룹을 저장할 리스트 초기화
    groups = []
    current_group = []

    # 각 행을 순회하면서 그룹을 만듭니다.
    for index, row in df.iterrows():
        if row['count'] == 1 and current_group:
            groups.append(pd.DataFrame(current_group))
            current_group = [row]
        else:
            current_group.append(row)

    # 마지막 그룹 추가 (남아있는 경우)
    if current_group:
        groups.append(pd.DataFrame(current_group))

    # 결과 저장할 리스트
    result_list = []

    # 각 그룹별로 passorfail 열의 숫자가 처음으로 1이 나오는 그룹과 그 다음 0이 연속되는 그룹으로 나누어 t-test를 수행합니다.
    for i, group in enumerate(groups):
        start_count_1 = None
        end_count_1 = None
        start_count_0 = None
        end_count_0 = None
        consecutive_count_1 = 0
        consecutive_count_0 = 0
        found_first_1 = False
        found_first_0 = False

        for index, row in group.iterrows():
            if row['passorfail'] == 1:
                if not found_first_1:
                    start_count_1 = row['count']
                    found_first_1 = True
                end_count_1 = row['count']
                consecutive_count_1 += 1
            elif found_first_1 and not found_first_0:
                start_count_0 = row['count']
                found_first_0 = True
                consecutive_count_0 += 1
                end_count_0 = row['count']
            elif found_first_0 and row['passorfail'] == 0:
                end_count_0 = row['count']
                consecutive_count_0 += 1

        # 그룹 나누기
        group_1 = group[(group['count'] >= start_count_1) & (group['count'] <= end_count_1)]
        group_0 = group[(group['count'] >= start_count_0) & (group['count'] <= end_count_0)]

        # t-test 수행
        significant_columns = []

        for col in columns_to_test:
            if not group_1[col].dropna().empty and not group_0[col].dropna().empty:
                t_stat, p_value = ttest_ind(group_1[col].dropna(), group_0[col].dropna(), equal_var=False)
                if p_value < 0.05:
                    significant_columns.append(col)

        # 결과 저장
        result_list.append({
            'Group': f'Group {i+1}',
            'Significant Columns': significant_columns,
            'Group 1 Means': group_1[columns_to_test].mean(),
            'Group 0 Means': group_0[columns_to_test].mean()
        })

    # 모든 그룹에서 유의미한 차이를 보인 공통 열들 확인
    common_significant_columns = set(result_list[0]['Significant Columns'])
    for result in result_list[1:]:
        common_significant_columns.intersection_update(result['Significant Columns'])

    # 다중 회귀 분석 수행
    if common_significant_columns:
        X = df[list(common_significant_columns)]
        y = df['passorfail']

        # 상수항 추가
        X = sm.add_constant(X)

        # 로지스틱 회귀 모델 피팅
        model = sm.Logit(y, X)
        result = model.fit()

        # 결과 요약 출력
        print(f"Group: {group_name}")
        print(result.summary())

        # 최적 수치 도출 (예: p-value가 0.05 이하인 변수들만 고려)
        significant_vars = result.pvalues[result.pvalues < 0.05].index
        significant_vars = significant_vars.drop('const', errors='ignore')
        print("Significant Variables: ", significant_vars)

        # 중요한 변수들의 평균 및 분산 계산
        important_vars_data = df[significant_vars]
        means = important_vars_data.mean()
        stds = important_vars_data.std()

        # 최적 범위 추정 (예: 평균 ± 1 표준편차)
        optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in significant_vars}
        print("Defect Ranges: ", optimal_ranges)

        print("\n")
    else:
        print(f"Group: {group_name} has no significant variables across all sub-groups.\n")


Group: 8917 has no significant variables across all sub-groups.

Group: 8412 has no significant variables across all sub-groups.

Group: 8722 has no significant variables across all sub-groups.

Group: 8573 has no significant variables across all sub-groups.

Group: 8413 has no significant variables across all sub-groups.

Group: 8600 has no significant variables across all sub-groups.

Group: 8576 has no significant variables across all sub-groups.



In [69]:
import pandas as pd
from scipy.stats import ttest_ind

# 그룹 데이터프레임 리스트
dfs = [df_8917, df_8412, df_8722, df_8573, df_8413, df_8600, df_8576]

# t-test를 수행할 열들
columns_to_test = ['molten_temp', 'facility_operation_cycleTime', 'production_cycletime', 'low_section_speed',
                   'high_section_speed', 'molten_volume', 'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
                   'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1', 'lower_mold_temp2', 'lower_mold_temp3',
                   'sleeve_temperature', 'physical_strength', 'Coolant_temperature', 'EMS_operation_time']

# 그룹별로 t-test 수행
for df in dfs:
    # 그룹 이름 추출
    group_name = df['mold_code'].iloc[0]

    # 그룹을 저장할 리스트 초기화
    groups = []
    current_group = []

    # 각 행을 순회하면서 그룹을 만듭니다.
    for index, row in df.iterrows():
        if row['count'] == 1 and current_group:
            groups.append(pd.DataFrame(current_group))
            current_group = [row]
        else:
            current_group.append(row)

    # 마지막 그룹 추가 (남아있는 경우)
    if current_group:
        groups.append(pd.DataFrame(current_group))

    # 연속적인 1과 0 데이터를 저장할 리스트 초기화
    group_1_all = []
    group_0_all = []

    for group in groups:
        start_count_1 = None
        end_count_1 = None
        start_count_0 = None
        end_count_0 = None
        found_first_1 = False
        found_first_0 = False

        for index, row in group.iterrows():
            if row['passorfail'] == 1:
                if not found_first_1:
                    start_count_1 = row['count']
                    found_first_1 = True
                end_count_1 = row['count']
            elif found_first_1 and not found_first_0:
                start_count_0 = row['count']
                found_first_0 = True
                end_count_0 = row['count']
            elif found_first_0 and row['passorfail'] == 0:
                end_count_0 = row['count']

        # 그룹 나누기
        group_1 = group[(group['count'] >= start_count_1) & (group['count'] <= end_count_1)]
        group_0 = group[(group['count'] >= start_count_0) & (group['count'] <= end_count_0)]

        group_1_all.append(group_1)
        group_0_all.append(group_0)

    # 그룹 데이터프레임 합치기
    group_1_combined = pd.concat(group_1_all, ignore_index=True)
    group_0_combined = pd.concat(group_0_all, ignore_index=True)

    # t-test 수행
    significant_columns = []

    for col in columns_to_test:
        if not group_1_combined[col].dropna().empty and not group_0_combined[col].dropna().empty:
            t_stat, p_value = ttest_ind(group_1_combined[col].dropna(), group_0_combined[col].dropna(), equal_var=False)
            if p_value < 0.05:
                significant_columns.append((col, t_stat, p_value))

    # 결과 출력
    print(f"df_{group_name} Significant Columns:")
    for col, t_stat, p_value in significant_columns:
        print(f"{col}: t-statistic = {t_stat}, p-value = {p_value}")

    print("\n")

# 모든 그룹에서 유의미한 차이를 보인 공통 열들 확인
common_significant_columns = set([col for col, _, _ in significant_columns])
for df in dfs:
    group_name = df['mold_code'].iloc[0]
    group_significant_columns = [col for col, _, _ in significant_columns if col in df.columns]
    common_significant_columns.intersection_update(group_significant_columns)

print("All Groups Common Significant Columns:")
print(common_significant_columns)


df_8917 Significant Columns:
molten_temp: t-statistic = 4.474951263377227, p-value = 7.663658518850572e-06
production_cycletime: t-statistic = -2.4889688993243038, p-value = 0.012815160896404299
low_section_speed: t-statistic = -8.927176137806928, p-value = 4.586357521195487e-19
cast_pressure: t-statistic = -20.906880480459712, p-value = 2.5311646156809974e-96
biscuit_thickness: t-statistic = 3.0789415205740998, p-value = 0.002079340362553751
upper_mold_temp1: t-statistic = 4.343652947586796, p-value = 1.4045449408986237e-05
upper_mold_temp2: t-statistic = -12.171278323950038, p-value = 5.119508425122805e-34
lower_mold_temp1: t-statistic = -15.709039972224923, p-value = 1.9249605010170332e-55
lower_mold_temp2: t-statistic = -2.0727715419957464, p-value = 0.03819978256961022
sleeve_temperature: t-statistic = 2.000518908286667, p-value = 0.04545042921668579
physical_strength: t-statistic = -7.3653636823257616, p-value = 1.8162939012371921e-13
Coolant_temperature: t-statistic = -21.060876

### 🅰️결론

프로세스 설명 및 각 단계에 대한 설명  
1. t-test 수행  
t-test는 두 그룹 간의 평균 차이를 검정하는 통계 방법입니다. 이 검정을 통해 각 그룹에서 연속적으로 나타나는 1(pass)와 0(fail) 데이터의 변수가 유의미하게 다른지 확인합니다. 이 단계는 다음과 같은 이유로 수행됩니다:  

목적: 그룹 간에 특정 변수들이 통계적으로 유의미한 차이를 보이는지 확인.  
이유: 유의미한 차이를 보이는 변수들은 품질에 영향을 미치는 중요한 요인일 가능성이 높습니다.  
t-test를 통해서 유의미한 변수들을 선정하고, 이는 다음 단계인 다중 회귀 분석에 입력 변수로 사용됩니다.  
  
-----
  
2. 다중 회귀 분석 수행  
다중 회귀 분석은 여러 독립 변수들이 종속 변수에 미치는 영향을 동시에 분석하는 통계 기법입니다. 여기서는 t-test로 선정된 유의미한 변수들을 독립 변수로 사용하여, passorfail (종속 변수)에 대한 영향을 평가합니다.  
  
목적: 여러 독립 변수들의 종속 변수에 대한 영향을 동시에 분석.  
이유: 독립 변수들이 상호작용할 수 있기 때문에, 이를 고려하여 각 변수의 실제 영향을 평가하기 위함.  
이 단계에서는 각 변수들의 회귀 계수와 유의미성(p-value)을 확인합니다. 유의미한 변수들은 품질에 중요한 영향을 미치는 변수들로 간주됩니다.  
  
----
  
3. 로지스틱 회귀 분석 수행 및 최적 범위 추정  
로지스틱 회귀 분석은 이진 종속 변수(여기서는 passorfail)를 예측하기 위해 사용되는 통계 모델입니다. 다중 회귀 분석을 통해 선정된 유의미한 변수들을 사용하여 로지스틱 회귀 모델을 구축합니다.  
목적: passorfail 결과를 예측하는 모델을 구축.  
이유: 품질 결과를 예측하는데 중요한 변수들을 최종적으로 식별하고, 이를 기반으로 최적 범위를 계산하기 위함.  
로지스틱 회귀 모델을 통해 최적 범위를 계산하는 과정은 다음과 같습니다:  
  
모델 피팅: 로지스틱 회귀 모델을 데이터에 피팅하여 각 변수의 회귀 계수와 유의미성(p-value)을 확인합니다.  
중요 변수 선정: 유의미한(p-value < 0.05) 변수들을 선정합니다.  
최적 범위 계산: 중요한 변수들의 평균과 표준편차를 계산하여, 평균 ± 1 표준편차의 범위를 최적 범위로 설정합니다. 이는 해당 변수들이 이 범위 내에 있을 때, pass 확률이 높아지는 경향이 있음을 의미합니다.  

In [73]:
import statsmodels.api as sm

# 그룹별로 작업 수행
for df in dfs:
    # 그룹 이름 추출
    group_name = df['mold_code'].iloc[0]

    # 그룹을 저장할 리스트 초기화
    groups = []
    current_group = []

    # 각 행을 순회하면서 그룹을 만듭니다.
    for index, row in df.iterrows():
        if row['count'] == 1 and current_group:
            groups.append(pd.DataFrame(current_group))
            current_group = [row]
        else:
            current_group.append(row)

    # 마지막 그룹 추가 (남아있는 경우)
    if current_group:
        groups.append(pd.DataFrame(current_group))

    # 연속적인 1과 0 데이터를 저장할 리스트 초기화
    group_1_all = []
    group_0_all = []

    for group in groups:
        start_count_1 = None
        end_count_1 = None
        start_count_0 = None
        end_count_0 = None
        found_first_1 = False
        found_first_0 = False

        for index, row in group.iterrows():
            if row['passorfail'] == 1:
                if not found_first_1:
                    start_count_1 = row['count']
                    found_first_1 = True
                end_count_1 = row['count']
            elif found_first_1 and not found_first_0:
                start_count_0 = row['count']
                found_first_0 = True
                end_count_0 = row['count']
            elif found_first_0 and row['passorfail'] == 0:
                end_count_0 = row['count']

        # 그룹 나누기
        group_1 = group[(group['count'] >= start_count_1) & (group['count'] <= end_count_1)]
        group_0 = group[(group['count'] >= start_count_0) & (group['count'] <= end_count_0)]

        group_1_all.append(group_1)
        group_0_all.append(group_0)

    # 그룹 데이터프레임 합치기
    group_1_combined = pd.concat(group_1_all, ignore_index=True)
    group_0_combined = pd.concat(group_0_all, ignore_index=True)

    # t-test 수행
    significant_columns = []

    for col in columns_to_test:
        if not group_1_combined[col].dropna().empty and not group_0_combined[col].dropna().empty:
            t_stat, p_value = ttest_ind(group_1_combined[col].dropna(), group_0_combined[col].dropna(), equal_var=False)
            if p_value < 0.05:
                significant_columns.append(col)

    if significant_columns:
        # 다중 회귀 분석 수행
        X = df[significant_columns]
        y = df['passorfail']

        # 상수항 추가
        X = sm.add_constant(X)

        # 로지스틱 회귀 모델 피팅
        model = sm.Logit(y, X)
        result = model.fit()

        # 결과 요약 출력
        print(f"Group: {group_name}")
        print(result.summary())

        # 중요한 변수들의 평균 및 분산 계산
        important_vars_data = df[significant_columns]
        means = important_vars_data.mean()
        stds = important_vars_data.std()

        # 최적 범위 추정 (예: 평균 ± 1 표준편차)
        optimal_ranges = {var: (means[var] - stds[var], means[var] + stds[var]) for var in significant_columns}
        print("Defect Ranges: ", optimal_ranges)
    else:
        print(f"Group: {group_name} has no significant variables.")


Optimization terminated successfully.
         Current function value: 0.046376
         Iterations 11
Group: 8917
                           Logit Regression Results                           
Dep. Variable:             passorfail   No. Observations:                31112
Model:                          Logit   Df Residuals:                    31099
Method:                           MLE   Df Model:                           12
Date:                Wed, 05 Jun 2024   Pseudo R-squ.:                  0.7400
Time:                        11:19:16   Log-Likelihood:                -1442.8
converged:                       True   LL-Null:                       -5549.7
Covariance Type:            nonrobust   LLR p-value:                     0.000
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   64.7709      8.730      7.419      0.000      47.66

## 🔫 tryshot 분석

In [64]:
df_tryshot = data_CK

In [65]:
df_tryshot.columns

Index(['time', 'date', 'count', 'working', 'molten_temp',
       'facility_operation_cycleTime', 'production_cycletime',
       'low_section_speed', 'high_section_speed', 'molten_volume',
       'cast_pressure', 'biscuit_thickness', 'upper_mold_temp1',
       'upper_mold_temp2', 'upper_mold_temp3', 'lower_mold_temp1',
       'lower_mold_temp2', 'lower_mold_temp3', 'sleeve_temperature',
       'physical_strength', 'Coolant_temperature', 'EMS_operation_time',
       'registration_time', 'passorfail', 'tryshot_signal', 'mold_code',
       'heating_furnace'],
      dtype='object')

In [66]:
# 필요한 패키지 불러오기
import pandas as pd
from scipy.stats import ttest_ind

# 데이터셋 정의
데이터셋 = df_tryshot

# mold_code 별로 분리
그룹별_데이터 = 데이터셋.groupby('mold_code')

# 각 mold_code에 대해 분석 진행
for mold_code, 데이터 in 그룹별_데이터:
    # D와 NaN 값으로 나누기
    D인_데이터 = 데이터[데이터['tryshot_signal'] == 'D']
    D가_아닌_데이터 = 데이터[데이터['tryshot_signal'].isnull() | (데이터['tryshot_signal'] != 'D')]
    
    # biscuit_thickness에 대한 유의미한 차이 확인
    t_stat_biscuit, p_value_biscuit = ttest_ind(D인_데이터['biscuit_thickness'].dropna(), D가_아닌_데이터['biscuit_thickness'].dropna())
    
    # molten_temp에 대한 유의미한 차이 확인
    t_stat_molten, p_value_molten = ttest_ind(D인_데이터['molten_temp'].dropna(), D가_아닌_데이터['molten_temp'].dropna())

    # p-value가 유의수준보다 작으면 차이가 유의미하다고 판단
    유의미한_차이_biscuit = p_value_biscuit < 0.05
    유의미한_차이_molten = p_value_molten < 0.05

    # 결과 출력
    print("mold_code:", mold_code)
    print("D인 그룹의 평균 biscuit 두께:", D인_데이터['biscuit_thickness'].mean())
    print("D가 아닌 그룹의 평균 biscuit 두께:", D가_아닌_데이터['biscuit_thickness'].mean())
    print("T-test 결과 (biscuit thickness) p-value:", p_value_biscuit)
    print("유의미한 차이가 있는가? (biscuit thickness)", "예" if 유의미한_차이_biscuit else "아니오")
    print("\n")
    
    print("D인 그룹의 평균 molten_temp:", D인_데이터['molten_temp'].mean())
    print("D가 아닌 그룹의 평균 molten_temp:", D가_아닌_데이터['molten_temp'].mean())
    print("T-test 결과 (molten_temp) p-value:", p_value_molten)
    print("유의미한 차이가 있는가? (molten_temp)", "예" if 유의미한_차이_molten else "아니오")
    print("\n")


mold_code: 8412
D인 그룹의 평균 biscuit 두께: 59.39549180327869
D가 아닌 그룹의 평균 biscuit 두께: 51.34075203736358
T-test 결과 (biscuit thickness) p-value: 4.993155777383443e-35
유의미한 차이가 있는가? (biscuit thickness) 예


D인 그룹의 평균 molten_temp: 730.5983606557377
D가 아닌 그룹의 평균 molten_temp: 725.1018053626749
T-test 결과 (molten_temp) p-value: 9.914500142264727e-20
유의미한 차이가 있는가? (molten_temp) 예


mold_code: 8413
D인 그룹의 평균 biscuit 두께: 70.45631067961165
D가 아닌 그룹의 평균 biscuit 두께: 47.43943033630748
T-test 결과 (biscuit thickness) p-value: 2.5820584984218466e-72
유의미한 차이가 있는가? (biscuit thickness) 예


D인 그룹의 평균 molten_temp: 727.2961165048544
D가 아닌 그룹의 평균 molten_temp: 723.4166094715168
T-test 결과 (molten_temp) p-value: 1.5672681624758897e-09
유의미한 차이가 있는가? (molten_temp) 예


mold_code: 8573
D인 그룹의 평균 biscuit 두께: nan
D가 아닌 그룹의 평균 biscuit 두께: 53.597019591496455
T-test 결과 (biscuit thickness) p-value: nan
유의미한 차이가 있는가? (biscuit thickness) 아니오


D인 그룹의 평균 molten_temp: nan
D가 아닌 그룹의 평균 molten_temp: 719.8491037932472
T-test 결과 (molten_

In [67]:
# 필요한 패키지 불러오기
import pandas as pd
from scipy.stats import ttest_ind

# 데이터셋 정의
데이터셋 = df_tryshot

# 결과를 저장할 빈 리스트 생성
결과 = []

# mold_code 별로 분리
그룹별_데이터 = 데이터셋.groupby('mold_code')

# 각 mold_code에 대해 분석 진행
for mold_code, 데이터 in 그룹별_데이터:
    # D와 NaN 값으로 나누기
    D인_데이터 = 데이터[데이터['tryshot_signal'] == 'D']
    D가_아닌_데이터 = 데이터[데이터['tryshot_signal'].isnull() | (데이터['tryshot_signal'] != 'D')]
    
    # biscuit_thickness에 대한 유의미한 차이 확인
    t_stat_biscuit, p_value_biscuit = ttest_ind(D인_데이터['biscuit_thickness'].dropna(), D가_아닌_데이터['biscuit_thickness'].dropna())
    
    # molten_temp에 대한 유의미한 차이 확인
    t_stat_molten, p_value_molten = ttest_ind(D인_데이터['molten_temp'].dropna(), D가_아닌_데이터['molten_temp'].dropna())

    # production_cycletime에 대한 유의미한 차이 확인
    t_stat_cycle, p_value_cycle = ttest_ind(D인_데이터['production_cycletime'].dropna(), D가_아닌_데이터['production_cycletime'].dropna())

    # p-value가 유의수준보다 작으면 차이가 유의미하다고 판단
    유의미한_차이_biscuit = p_value_biscuit < 0.05
    유의미한_차이_molten = p_value_molten < 0.05
    유의미한_차이_cycle = p_value_cycle < 0.05

    # 결과 저장
    결과.append({
        'mold_code': mold_code,
        'biscuit_thickness 차이 있는가': "예" if 유의미한_차이_biscuit else "아니오",
        'molten_temp 차이 있는가': "예" if 유의미한_차이_molten else "아니오",
        'production_cycletime 차이 있는가': "예" if 유의미한_차이_cycle else "아니오"
    })

# 결과를 데이터프레임으로 변환하여 출력
결과_df = pd.DataFrame(결과)

# 결과에서 조심해서 볼것: 8573, 8600은 tryshot 없음!! -> 따라서 결과 출력에서 제외

print('💡8573, 8600은 tryshot 없으므로 출력 X')

# mold_code가 8573이거나 8600인 행 제거
결과_df = 결과_df[~결과_df['mold_code'].isin(['8573', '8600'])]

결과_df

💡8573, 8600은 tryshot 없으므로 출력 X


,mold_code,biscuit_thickness 차이 있는가,molten_temp 차이 있는가,production_cycletime 차이 있는가
0,8412,예,예,예
1,8413,예,예,예
2,8573,아니오,아니오,아니오
3,8576,예,아니오,예
4,8600,아니오,아니오,아니오
5,8722,예,아니오,예
6,8917,예,예,예
